In [ ]:
import SBFVAR
import pandas as pd
import numpy as np
import pickle


# Preparations
#---------------------

io_data = "/home/u80856195/git/MUFBVAR-master/examples/hist.xlsx"

#Model Specification
H = 96          # forecast horizon
nsim = 20      # number of draws from Posterior Density
nburn = 0.5     # number of draws to discard
nlags = 12   # Number of lags
thining = 1     # Thining 

hyp = [0.09, 4.3, 1, 2.7, 4.3] # Hyperparameters see documentation for details

frequencies = ["Q","M","W"] # Frequencies




# Load the data
# --------------
data = []
for freq in range(len(frequencies)):
        freq = frequencies[freq]
        data_temp = pd.read_excel(io_data, sheet_name = freq, index_col = 0)
        data.append(data_temp)

#Transformations
trans = [np.array((1,1)), np.array((1,1,1)), np.array((1,1,1))]    


# Initialize data class            
mufbvar_data = SBFVAR.mufbvar_data(data, trans, frequencies)


# Fit and Forecast
#--------------------

# Initialize model class    
model =  SBFVAR.multifrequency_var(nsim, nburn, nlags, thining)

DEBUG INFO - Variable counts by frequency:
  Q variables: 2
  M variables: 3
  W variables: 3
DEBUG INFO - Combined variable lists:
  varlist_list[0]: 5 variables
    ['m_1' 'm_2' 'm_3' 'q_1' 'q_2']
  varlist_list[1]: 8 variables
    ['w_1' 'w_2' 'w_3' 'm_1' 'm_2' 'm_3' 'q_1' 'q_2']


In [ ]:
self = model

In [ ]:
var_of_interest=None
temp_agg='mean'

In [ ]:
    explosive_counter = 0
    valid_draws = []
    
    self.nex = 1
    self.hyp = hyp
    self.temp_agg = temp_agg
    self.var_of_interest = var_of_interest
    
    assert self.temp_agg in ("mean", "sum"), f"Invalid temp_agg: {self.temp_agg}. Choose 'mean' or 'sum'."
    
    # Data from mufbvar_data
    YMX_list = copy.deepcopy(mufbvar_data.YMX_list)
    YM0_list = copy.deepcopy(mufbvar_data.YM0_list)
    select_m_list = copy.deepcopy(mufbvar_data.select_m_list)
    vars_m_list = copy.deepcopy(mufbvar_data.vars_m_list)
    YMh_list = copy.deepcopy(mufbvar_data.YMh_list)
    index_list = copy.deepcopy(mufbvar_data.index_list)
    frequencies = copy.deepcopy(mufbvar_data.frequencies)
    self.frequencies = frequencies
    YQX_list = copy.deepcopy(mufbvar_data.YQX_list)
    YQ0_list = copy.deepcopy(mufbvar_data.YQ0_list)
    select_q = copy.deepcopy(mufbvar_data.select_q)
    input_data_Q = copy.deepcopy(mufbvar_data.input_data_Q)
    self.input_data_Q = input_data_Q
    varlist_list = copy.deepcopy(mufbvar_data.varlist_list[-1])
    select_list = copy.deepcopy(mufbvar_data.select_list)
    select_c_list = copy.deepcopy(mufbvar_data.select_c_list)
    Nm_list = copy.deepcopy(mufbvar_data.Nm_list)
    nv_list = copy.deepcopy(mufbvar_data.nv_list)
    Nq_list = copy.deepcopy(mufbvar_data.Nq_list)
    select_list_sep = copy.deepcopy(mufbvar_data.select_list_sep)
    freq_ratio_list = copy.deepcopy(mufbvar_data.freq_ratio_list)
    YQ_list = copy.deepcopy(mufbvar_data.YQ_list)
    Tstar_list = copy.deepcopy(mufbvar_data.Tstar_list)
    T_list = copy.deepcopy(mufbvar_data.T_list)
    YDATA_list = copy.deepcopy(mufbvar_data.YDATA_list)
    YM_list = copy.deepcopy(mufbvar_data.YM_list)
    input_data = copy.deepcopy(mufbvar_data.input_data)
    self.input_data = input_data

    nburn = round((self.nburn_perc)*math.ceil(self.nsim/self.thining))
    self.nburn = nburn
    
    nlags = self.nlags
    p = self.nlags  # Number of lags for VAR
    
    # Validate frequency ratios
    rmw = freq_ratio_list[1]  # Monthly to weekly ratio
    rqw = freq_ratio_list[0] * rmw  # Quarterly to weekly ratio
    assert rmw == 4, "Monthly aggregation requires exactly 4 weeks/month"
    assert rqw == 12, "Quarterly aggregation requires exactly 12 weeks/quarter"
    
    # Extract variable counts
    Nq = Nq_list[0]  # Quarterly variables
    Nm = Nm_list[0]  # Monthly variables
    Nw = Nm_list[1]  # Weekly variables
    Ntotal = Nq + Nm + Nw  # Total number of variables across all frequencies
    
    # Extract data for each frequency
    YQ = copy.deepcopy(YQ_list[0])  # Quarterly data
    YM = copy.deepcopy(YM_list[0])  # Monthly data
    YW = copy.deepcopy(YM_list[1])  # Weekly data (second entry in YM_list)
    
    # Get observation counts
    Tw = YW.shape[0]  # Total weekly observations
    Tm = YM.shape[0]  # Monthly observations
    Tq = YQ.shape[0]  # Quarterly observations
    
    # Print data dimensions for verification
    print(f"Data dimensions - Weekly: {YW.shape}, Monthly: {YM.shape}, Quarterly: {YQ.shape}")
    
    # Number of observations after burn-in (T0 = initial lag period)
    T0 = int(nlags)  # Initial observations used for lags
    nobs = Tw - T0  
    
    # STATE SPACE self STRUCTURE
    # ---------------------------
    
    # State vector structure includes all variables in VAR
    var_block = Ntotal * p  # All variables with lags for VAR
    monthly_latent_block = Nm * rmw  # Month of weekly latent states for monthly vars
    quarterly_latent_block = Nq * rqw  # Quarter of weekly latent states for quarterly vars
    
    # Define indices within state vector
    monthly_start = var_block
    quarterly_start = monthly_start + monthly_latent_block
    
    # Total state vector size
    nstate = var_block + monthly_latent_block + quarterly_latent_block
    
    # Print state vector structure details
    print(f"State vector structure:")
    print(f"  Variables block (with lags): {var_block} states")
    print(f"  Monthly latent states block: {monthly_latent_block} states (starting at index {monthly_start})")
    print(f"  Quarterly latent states block: {quarterly_latent_block} states (starting at index {quarterly_start})")
    print(f"  Total state vector size: {nstate} states")
    
    # Initialize matrices for MCMC sampling
    Sigmap = np.zeros((math.ceil((self.nsim)/self.thining), Ntotal, Ntotal))
    Phip = np.zeros((math.ceil((self.nsim)/self.thining), Ntotal*p+1, Ntotal))  # All variables
    Cons = np.zeros((math.ceil((self.nsim)/self.thining), Ntotal))  # Constants for all variables
    
    # Initialize transition matrix F
    F = np.zeros((nstate, nstate))
    
    # Initialize VAR coefficients for all variables
    Phi = np.vstack((0.95 * np.eye(Ntotal), np.zeros((Ntotal*(p-1), Ntotal)), np.zeros((1, Ntotal))))  # Last row for constant
    
    # Set VAR dynamics in transition matrix
    F[:Ntotal, :Ntotal*p] = Phi[:-1, :].T  # VAR coefficients (excluding constant)
    
    # Lag-shifting section for all variables
    for i in range(p-1):
        F[Ntotal*(i+1):Ntotal*(i+2), Ntotal*i:Ntotal*(i+1)] = np.eye(Ntotal)
    
    # Monthly latent states: shifting blocks with weekly influence
    # Week 1 is influenced by weekly variables (10%)
    F[monthly_start:monthly_start+Nm, :Nw] = 0.1 * np.eye(Nm, Nw)
    
    # Shift weeks within each month, creating a diagonal structure
    for i in range(rmw-1):
        src_pos = monthly_start + i*Nm
        dest_pos = monthly_start + (i+1)*Nm
        F[dest_pos:dest_pos+Nm, src_pos:src_pos+Nm] = np.eye(Nm)
    
    # Quarterly latent states: shifting blocks with weekly influence
    # Week 1 is influenced by weekly variables (5%)
    F[quarterly_start:quarterly_start+Nq, :Nw] = 0.05 * np.eye(Nq, Nw)
    
    # Shift weeks within each quarter
    for i in range(rqw-1):
        src_pos = quarterly_start + i*Nq
        dest_pos = quarterly_start + (i+1)*Nq
        F[dest_pos:dest_pos+Nq, src_pos:src_pos+Nq] = np.eye(Nq)
    
    # Constant term vector
    c = np.zeros((nstate, 1))
    c[:Ntotal] = np.atleast_2d(Phi[-1, :]).T  # Constants for all variables
    
    # MEASUREMENT EQUATIONS WITH ENHANCED CONSTRAINTS
    # ---------------------------------------------
    
    # Weekly measurement (always available)
    H_w = np.zeros((Nw, nstate))
    H_w[:, :Nw] = np.eye(Nw)  # Directly observe first Nw state elements
    
    # Monthly measurement (available at month-end)
    H_m = np.zeros((Nm, nstate))
    # For monthly variables, aggregate all weekly latent states
    for m in range(Nm):
        for w in range(rmw):
            state_idx = monthly_start + w*Nm + m
            if self.temp_agg == 'mean':
                H_m[m, state_idx] = 1.0/rmw  # Average of weeks in month
            else:  # 'sum'
                H_m[m, state_idx] = 1.0  # Sum of weeks in month
    
    # Quarterly measurement (available at quarter-end)
    H_q = np.zeros((Nq, nstate))
    # For quarterly variables, aggregate all weekly latent states
    for q in range(Nq):
        for w in range(rqw):
            state_idx = quarterly_start + w*Nq + q
            if self.temp_agg == 'mean':
                H_q[q, state_idx] = 1.0/rqw  # Average of weeks in quarter
            else:  # 'sum'
                H_q[q, state_idx] = 1.0  # Sum of weeks in quarter
    
    # Add direct constraints to also enforce VAR block alignment
    # This creates a stronger link between VAR variables and latent states
    H_m_constraint = np.zeros((Nm, nstate))
    H_q_constraint = np.zeros((Nq, nstate))
    
    # Link monthly variables in VAR block with first week of latent states
    for m in range(Nm):
        # VAR block for monthly variable
        H_m_constraint[m, Nw+m] = 1.0
        # First week latent state
        H_m_constraint[m, monthly_start+m] = -1.0
    
    # Link quarterly variables in VAR block with first week of latent states
    for q in range(Nq):
        # VAR block for quarterly variable
        H_q_constraint[q, Nw+Nm+q] = 1.0
        # First week latent state
        H_q_constraint[q, quarterly_start+q] = -1.0
    
    # Print measurement matrix details for verification
    print("\nMeasurement equations:")
    print(f"Weekly measurement: Using {Nw} direct weekly observations")
    for m in range(min(2, Nm)):
        active_indices = np.where(H_m[m] != 0)[0]
        print(f"Monthly var {m+1}: Aggregating {len(active_indices)} weekly latent states with weights {H_m[m, active_indices[0]]:.6f}")
        
    for q in range(min(2, Nq)):
        active_indices = np.where(H_q[q] != 0)[0]
        print(f"Quarterly var {q+1}: Aggregating {len(active_indices)} weekly latent states with weights {H_q[q, active_indices[0]]:.6f}")
    
    print("Added VAR-to-latent state alignment constraints")
    
    # SYSTEM NOISE AND INITIALIZATION
    # ------------------------------
    
    # System noise affects VAR variables differently than latent states
    Q = np.zeros((nstate, nstate))
    
    # VAR block: moderate process noise
    Q[:Ntotal, :Ntotal] = 1e-4 * np.eye(Ntotal)
    
    # Monthly latent states: significantly lower process noise
    for m in range(Nm):
        for w in range(rmw):
            idx = monthly_start + w*Nm + m
            Q[idx, idx] = 1e-6  # Reduced process noise for smoother weekly patterns
    
    # Quarterly latent states: lowest process noise
    for q in range(Nq):
        for w in range(rqw):
            idx = quarterly_start + w*Nq + q
            Q[idx, idx] = 1e-7  # Even lower process noise for quarterly patterns
    
    # Initialize state vector
    a_t = np.zeros(nstate)
    P_t = np.eye(nstate) * 1e-3  # Initialize P_t here first

    # Higher uncertainty for VAR block
    P_t[:Ntotal, :Ntotal] = np.eye(Ntotal) * 1e-2
    # Initialize weekly variables with available data
    if YW.shape[0] > 0:
        a_t[:Nw] = YW[0, :Nw]
        print(f"Weekly vars initialized with first observation")
    
    # Initialize monthly variables in VAR block
    if YM.shape[0] > 0:
        a_t[Nw:Nw+Nm] = YM[0, :]
    
    # Initialize quarterly variables in VAR block
    if YQ.shape[0] > 0:
        a_t[Nw+Nm:Ntotal] = YQ[0, :]
    
    # IMPROVED INITIALIZATION FOR LATENT STATES
    # Initialize monthly latent states with reasonable patterns
    if YM.shape[0] > 0:
        for m in range(Nm):
            m_value = YM[0, m]
            # Create a realistic weekly pattern around the observed value
            for w in range(rmw):
                # Small variations (±5%)
                a_t[monthly_start + w*Nm + m] = m_value * (0.95 + 0.1 * (w / (rmw-1)))
                # Lower initial uncertainty
                P_t[monthly_start + w*Nm + m, monthly_start + w*Nm + m] = 0.01
            print(f"Monthly var {m+1} initialized with more stable weekly pattern")
    
    # Initialize quarterly latent states with reasonable patterns
    if YQ.shape[0] > 0:
        for q in range(Nq):
            q_value = YQ[0, q]
            # Create a realistic weekly pattern for quarterly data
            for w in range(rqw):
                # Gentle seasonal pattern
                a_t[quarterly_start + w*Nq + q] = q_value * (0.9 + 0.2 * np.sin(np.pi * w / rqw))
                # Lower initial uncertainty
                P_t[quarterly_start + w*Nq + q, quarterly_start + w*Nq + q] = 0.01
            print(f"Quarterly var {q+1} initialized with more stable weekly pattern")
    
    # PREPARE KALMAN FILTER DATA
    # -------------------------
    
    # Prepare observed data vectors with clear frequency markers
    q_obs_periods = np.zeros(Tw, dtype=bool)  # Quarters observed
    m_obs_periods = np.zeros(Tw, dtype=bool)  # Months observed
    
    # Mark which weekly periods have quarterly/monthly observations
    # Quarterly data is available at the end of each quarter
    for t in range(rqw-1, Tw, rqw):
        q_idx = t // rqw
        if q_idx < Tq:
            q_obs_periods[t] = True
    
    # Monthly data is available at the end of each month
    for t in range(rmw-1, Tw, rmw):
        m_idx = t // rmw
        if m_idx < Tm:
            m_obs_periods[t] = True
    
    # Storage for filtered and smoothed states
    a_filtered = np.zeros((nobs, nstate))
    P_filtered = np.zeros((nobs, nstate, nstate))
    a_draws = np.zeros((self.nsim, nobs, nstate))
    

    

NameError: name 'copy' is not defined

In [ ]:
import os
import sys
import numpy as np
import math
from collections import deque
from scipy.linalg import companion
from scipy.stats import invwishart
import pandas as pd
from scipy.stats import multivariate_normal
from datetime import datetime
from pandas.tseries.offsets import Week, MonthBegin, QuarterBegin, Day
import itertools
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import partial
import matplotlib.backends.backend_pdf
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import pickle
import copy
from SBFVAR.cholcov.cholcov_module import cholcovOrEigendecomp
from SBFVAR.inverse.matrix_inversion import invert_matrix
from SBFVAR.mfbvar_funcs import calc_yyact, is_explosive


In [ ]:
    explosive_counter = 0
    valid_draws = []
    
    self.nex = 1
    self.hyp = hyp
    self.temp_agg = temp_agg
    self.var_of_interest = var_of_interest
    
    assert self.temp_agg in ("mean", "sum"), f"Invalid temp_agg: {self.temp_agg}. Choose 'mean' or 'sum'."
    
    # Data from mufbvar_data
    YMX_list = copy.deepcopy(mufbvar_data.YMX_list)
    YM0_list = copy.deepcopy(mufbvar_data.YM0_list)
    select_m_list = copy.deepcopy(mufbvar_data.select_m_list)
    vars_m_list = copy.deepcopy(mufbvar_data.vars_m_list)
    YMh_list = copy.deepcopy(mufbvar_data.YMh_list)
    index_list = copy.deepcopy(mufbvar_data.index_list)
    frequencies = copy.deepcopy(mufbvar_data.frequencies)
    self.frequencies = frequencies
    YQX_list = copy.deepcopy(mufbvar_data.YQX_list)
    YQ0_list = copy.deepcopy(mufbvar_data.YQ0_list)
    select_q = copy.deepcopy(mufbvar_data.select_q)
    input_data_Q = copy.deepcopy(mufbvar_data.input_data_Q)
    self.input_data_Q = input_data_Q
    varlist_list = copy.deepcopy(mufbvar_data.varlist_list[-1])
    select_list = copy.deepcopy(mufbvar_data.select_list)
    select_c_list = copy.deepcopy(mufbvar_data.select_c_list)
    Nm_list = copy.deepcopy(mufbvar_data.Nm_list)
    nv_list = copy.deepcopy(mufbvar_data.nv_list)
    Nq_list = copy.deepcopy(mufbvar_data.Nq_list)
    select_list_sep = copy.deepcopy(mufbvar_data.select_list_sep)
    freq_ratio_list = copy.deepcopy(mufbvar_data.freq_ratio_list)
    YQ_list = copy.deepcopy(mufbvar_data.YQ_list)
    Tstar_list = copy.deepcopy(mufbvar_data.Tstar_list)
    T_list = copy.deepcopy(mufbvar_data.T_list)
    YDATA_list = copy.deepcopy(mufbvar_data.YDATA_list)
    YM_list = copy.deepcopy(mufbvar_data.YM_list)
    input_data = copy.deepcopy(mufbvar_data.input_data)
    self.input_data = input_data

    nburn = round((self.nburn_perc)*math.ceil(self.nsim/self.thining))
    self.nburn = nburn
    
    nlags = self.nlags
    p = self.nlags  # Number of lags for VAR
    
    # Validate frequency ratios
    rmw = freq_ratio_list[1]  # Monthly to weekly ratio
    rqw = freq_ratio_list[0] * rmw  # Quarterly to weekly ratio
    assert rmw == 4, "Monthly aggregation requires exactly 4 weeks/month"
    assert rqw == 12, "Quarterly aggregation requires exactly 12 weeks/quarter"
    
    # Extract variable counts
    Nq = Nq_list[0]  # Quarterly variables
    Nm = Nm_list[0]  # Monthly variables
    Nw = Nm_list[1]  # Weekly variables
    Ntotal = Nq + Nm + Nw  # Total number of variables across all frequencies
    
    # Extract data for each frequency
    YQ = copy.deepcopy(YQ_list[0])  # Quarterly data
    YM = copy.deepcopy(YM_list[0])  # Monthly data
    YW = copy.deepcopy(YM_list[1])  # Weekly data (second entry in YM_list)
    
    # Get observation counts
    Tw = YW.shape[0]  # Total weekly observations
    Tm = YM.shape[0]  # Monthly observations
    Tq = YQ.shape[0]  # Quarterly observations
    
    # Print data dimensions for verification
    print(f"Data dimensions - Weekly: {YW.shape}, Monthly: {YM.shape}, Quarterly: {YQ.shape}")
    
    # Number of observations after burn-in (T0 = initial lag period)
    T0 = int(nlags)  # Initial observations used for lags
    nobs = Tw - T0  
    
    # STATE SPACE self STRUCTURE
    # ---------------------------
    
    # State vector structure includes all variables in VAR
    var_block = Ntotal * p  # All variables with lags for VAR
    monthly_latent_block = Nm * rmw  # Month of weekly latent states for monthly vars
    quarterly_latent_block = Nq * rqw  # Quarter of weekly latent states for quarterly vars
    
    # Define indices within state vector
    monthly_start = var_block
    quarterly_start = monthly_start + monthly_latent_block
    
    # Total state vector size
    nstate = var_block + monthly_latent_block + quarterly_latent_block
    
    # Print state vector structure details
    print(f"State vector structure:")
    print(f"  Variables block (with lags): {var_block} states")
    print(f"  Monthly latent states block: {monthly_latent_block} states (starting at index {monthly_start})")
    print(f"  Quarterly latent states block: {quarterly_latent_block} states (starting at index {quarterly_start})")
    print(f"  Total state vector size: {nstate} states")
    
    # Initialize matrices for MCMC sampling
    Sigmap = np.zeros((math.ceil((self.nsim)/self.thining), Ntotal, Ntotal))
    Phip = np.zeros((math.ceil((self.nsim)/self.thining), Ntotal*p+1, Ntotal))  # All variables
    Cons = np.zeros((math.ceil((self.nsim)/self.thining), Ntotal))  # Constants for all variables
    
    # Initialize transition matrix F
    F = np.zeros((nstate, nstate))
    
    # Initialize VAR coefficients for all variables
    Phi = np.vstack((0.95 * np.eye(Ntotal), np.zeros((Ntotal*(p-1), Ntotal)), np.zeros((1, Ntotal))))  # Last row for constant
    
    # Set VAR dynamics in transition matrix
    F[:Ntotal, :Ntotal*p] = Phi[:-1, :].T  # VAR coefficients (excluding constant)
    
    # Lag-shifting section for all variables
    for i in range(p-1):
        F[Ntotal*(i+1):Ntotal*(i+2), Ntotal*i:Ntotal*(i+1)] = np.eye(Ntotal)
    
    # Monthly latent states: shifting blocks with weekly influence
    # Week 1 is influenced by weekly variables (10%)
    F[monthly_start:monthly_start+Nm, :Nw] = 0.1 * np.eye(Nm, Nw)
    
    # Shift weeks within each month, creating a diagonal structure
    for i in range(rmw-1):
        src_pos = monthly_start + i*Nm
        dest_pos = monthly_start + (i+1)*Nm
        F[dest_pos:dest_pos+Nm, src_pos:src_pos+Nm] = np.eye(Nm)
    
    # Quarterly latent states: shifting blocks with weekly influence
    # Week 1 is influenced by weekly variables (5%)
    F[quarterly_start:quarterly_start+Nq, :Nw] = 0.05 * np.eye(Nq, Nw)
    
    # Shift weeks within each quarter
    for i in range(rqw-1):
        src_pos = quarterly_start + i*Nq
        dest_pos = quarterly_start + (i+1)*Nq
        F[dest_pos:dest_pos+Nq, src_pos:src_pos+Nq] = np.eye(Nq)
    
    # Constant term vector
    c = np.zeros((nstate, 1))
    c[:Ntotal] = np.atleast_2d(Phi[-1, :]).T  # Constants for all variables
    
    # MEASUREMENT EQUATIONS WITH ENHANCED CONSTRAINTS
    # ---------------------------------------------
    
    # Weekly measurement (always available)
    H_w = np.zeros((Nw, nstate))
    H_w[:, :Nw] = np.eye(Nw)  # Directly observe first Nw state elements
    
    # Monthly measurement (available at month-end)
    H_m = np.zeros((Nm, nstate))
    # For monthly variables, aggregate all weekly latent states
    for m in range(Nm):
        for w in range(rmw):
            state_idx = monthly_start + w*Nm + m
            if self.temp_agg == 'mean':
                H_m[m, state_idx] = 1.0/rmw  # Average of weeks in month
            else:  # 'sum'
                H_m[m, state_idx] = 1.0  # Sum of weeks in month
    
    # Quarterly measurement (available at quarter-end)
    H_q = np.zeros((Nq, nstate))
    # For quarterly variables, aggregate all weekly latent states
    for q in range(Nq):
        for w in range(rqw):
            state_idx = quarterly_start + w*Nq + q
            if self.temp_agg == 'mean':
                H_q[q, state_idx] = 1.0/rqw  # Average of weeks in quarter
            else:  # 'sum'
                H_q[q, state_idx] = 1.0  # Sum of weeks in quarter
    
    # Add direct constraints to also enforce VAR block alignment
    # This creates a stronger link between VAR variables and latent states
    H_m_constraint = np.zeros((Nm, nstate))
    H_q_constraint = np.zeros((Nq, nstate))
    
    # Link monthly variables in VAR block with first week of latent states
    for m in range(Nm):
        # VAR block for monthly variable
        H_m_constraint[m, Nw+m] = 1.0
        # First week latent state
        H_m_constraint[m, monthly_start+m] = -1.0
    
    # Link quarterly variables in VAR block with first week of latent states
    for q in range(Nq):
        # VAR block for quarterly variable
        H_q_constraint[q, Nw+Nm+q] = 1.0
        # First week latent state
        H_q_constraint[q, quarterly_start+q] = -1.0
    
    # Print measurement matrix details for verification
    print("\nMeasurement equations:")
    print(f"Weekly measurement: Using {Nw} direct weekly observations")
    for m in range(min(2, Nm)):
        active_indices = np.where(H_m[m] != 0)[0]
        print(f"Monthly var {m+1}: Aggregating {len(active_indices)} weekly latent states with weights {H_m[m, active_indices[0]]:.6f}")
        
    for q in range(min(2, Nq)):
        active_indices = np.where(H_q[q] != 0)[0]
        print(f"Quarterly var {q+1}: Aggregating {len(active_indices)} weekly latent states with weights {H_q[q, active_indices[0]]:.6f}")
    
    print("Added VAR-to-latent state alignment constraints")
    
    # SYSTEM NOISE AND INITIALIZATION
    # ------------------------------
    
    # System noise affects VAR variables differently than latent states
    Q = np.zeros((nstate, nstate))
    
    # VAR block: moderate process noise
    Q[:Ntotal, :Ntotal] = 1e-4 * np.eye(Ntotal)
    
    # Monthly latent states: significantly lower process noise
    for m in range(Nm):
        for w in range(rmw):
            idx = monthly_start + w*Nm + m
            Q[idx, idx] = 1e-6  # Reduced process noise for smoother weekly patterns
    
    # Quarterly latent states: lowest process noise
    for q in range(Nq):
        for w in range(rqw):
            idx = quarterly_start + w*Nq + q
            Q[idx, idx] = 1e-7  # Even lower process noise for quarterly patterns
    
    # Initialize state vector
    a_t = np.zeros(nstate)
    P_t = np.eye(nstate) * 1e-3  # Initialize P_t here first

    # Higher uncertainty for VAR block
    P_t[:Ntotal, :Ntotal] = np.eye(Ntotal) * 1e-2
    # Initialize weekly variables with available data
    if YW.shape[0] > 0:
        a_t[:Nw] = YW[0, :Nw]
        print(f"Weekly vars initialized with first observation")
    
    # Initialize monthly variables in VAR block
    if YM.shape[0] > 0:
        a_t[Nw:Nw+Nm] = YM[0, :]
    
    # Initialize quarterly variables in VAR block
    if YQ.shape[0] > 0:
        a_t[Nw+Nm:Ntotal] = YQ[0, :]
    
    # IMPROVED INITIALIZATION FOR LATENT STATES
    # Initialize monthly latent states with reasonable patterns
    if YM.shape[0] > 0:
        for m in range(Nm):
            m_value = YM[0, m]
            # Create a realistic weekly pattern around the observed value
            for w in range(rmw):
                # Small variations (±5%)
                a_t[monthly_start + w*Nm + m] = m_value * (0.95 + 0.1 * (w / (rmw-1)))
                # Lower initial uncertainty
                P_t[monthly_start + w*Nm + m, monthly_start + w*Nm + m] = 0.01
            print(f"Monthly var {m+1} initialized with more stable weekly pattern")
    
    # Initialize quarterly latent states with reasonable patterns
    if YQ.shape[0] > 0:
        for q in range(Nq):
            q_value = YQ[0, q]
            # Create a realistic weekly pattern for quarterly data
            for w in range(rqw):
                # Gentle seasonal pattern
                a_t[quarterly_start + w*Nq + q] = q_value * (0.9 + 0.2 * np.sin(np.pi * w / rqw))
                # Lower initial uncertainty
                P_t[quarterly_start + w*Nq + q, quarterly_start + w*Nq + q] = 0.01
            print(f"Quarterly var {q+1} initialized with more stable weekly pattern")
    
    # PREPARE KALMAN FILTER DATA
    # -------------------------
    
    # Prepare observed data vectors with clear frequency markers
    q_obs_periods = np.zeros(Tw, dtype=bool)  # Quarters observed
    m_obs_periods = np.zeros(Tw, dtype=bool)  # Months observed
    
    # Mark which weekly periods have quarterly/monthly observations
    # Quarterly data is available at the end of each quarter
    for t in range(rqw-1, Tw, rqw):
        q_idx = t // rqw
        if q_idx < Tq:
            q_obs_periods[t] = True
    
    # Monthly data is available at the end of each month
    for t in range(rmw-1, Tw, rmw):
        m_idx = t // rmw
        if m_idx < Tm:
            m_obs_periods[t] = True
    
    # Storage for filtered and smoothed states
    a_filtered = np.zeros((nobs, nstate))
    P_filtered = np.zeros((nobs, nstate, nstate))
    a_draws = np.zeros((self.nsim, nobs, nstate))
    

    

Data dimensions - Weekly: (947, 3), Monthly: (236, 3), Quarterly: (234, 2)
State vector structure:
  Variables block (with lags): 96 states
  Monthly latent states block: 12 states (starting at index 96)
  Quarterly latent states block: 24 states (starting at index 108)
  Total state vector size: 132 states

Measurement equations:
Weekly measurement: Using 3 direct weekly observations
Monthly var 1: Aggregating 4 weekly latent states with weights 0.250000
Monthly var 2: Aggregating 4 weekly latent states with weights 0.250000
Quarterly var 1: Aggregating 12 weekly latent states with weights 0.083333
Quarterly var 2: Aggregating 12 weekly latent states with weights 0.083333
Added VAR-to-latent state alignment constraints
Weekly vars initialized with first observation
Monthly var 1 initialized with more stable weekly pattern
Monthly var 2 initialized with more stable weekly pattern
Monthly var 3 initialized with more stable weekly pattern
Quarterly var 1 initialized with more stable week

In [ ]:
j = 0

In [ ]:
        if j > 0:
            a_t = a_draws[j-1, -1]
            P_t = P_filtered[-1]
        
        # Kalman filter loop through all periods
        for t in range(nobs):
            # Current week index
            w_idx = T0 + t
            
            # PREDICTION STEP
            # --------------
            a_pred = F @ a_t + c.flatten()
            P_pred = F @ P_t @ F.T + Q
            P_pred = 0.5 * (P_pred + P_pred.T)  # Ensure perfect symmetry
            
            # DETERMINE AVAILABLE OBSERVATIONS
            # -----------------------------
            
            # Check which observations are available at this period
            is_quarter_end = q_obs_periods[w_idx] if w_idx < len(q_obs_periods) else False
            is_month_end = m_obs_periods[w_idx] if w_idx < len(m_obs_periods) else False
            
            # Get indices for the observation vectors
            q_idx = w_idx // rqw if is_quarter_end else -1
            m_idx = w_idx // rmw if is_month_end else -1
            
            # Debug output for first few periods
            if j == 0 and t < 5:
                print(f"Period {t+1}: week_idx={w_idx}, quarter_end={is_quarter_end}, month_end={is_month_end}")
                if is_quarter_end:
                    print(f"  Quarterly observation: {YQ[q_idx]}")
                if is_month_end:
                    print(f"  Monthly observation: {YM[m_idx]}")
            
            # BUILD ENHANCED MEASUREMENT MATRICES AND OBSERVATION VECTOR
            # -----------------------------------------------------
            
            # Start with weekly observations (always available)
            H_matrices = [H_w]
            y_obs = [YW[w_idx]]
            
            # Add monthly observations if available
            monthly_constraints_added = False
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                H_matrices.append(H_m)
                y_obs.append(YM[m_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))  # Zero difference means equality constraint
                monthly_constraints_added = True
            
            # Add quarterly observations if available
            quarterly_constraints_added = False
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                H_matrices.append(H_q)
                y_obs.append(YQ[q_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))  # Zero difference means equality constraint
                quarterly_constraints_added = True
            
            # Also enforce VAR-latent alignment constraints at every period
            # (not just at month/quarter end)
            if not monthly_constraints_added and t % 5 == 0:  # Every 5 periods to avoid too much constraint
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))
            
            if not quarterly_constraints_added and t % 10 == 0:  # Less frequent for quarterly
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))
            
            # Combined measurement matrix and observation vector
            H = np.vstack(H_matrices)
            y = np.concatenate(y_obs)
            
            # DIFFERENT MEASUREMENT NOISE BY FREQUENCY - EXTREMELY PRECISE FOR CONSTRAINTS
            # ---------------------------------------------------------------------
            
            # Create measurement noise matrix with precision scaled by frequency
            R = np.zeros((len(y), len(y)))
            
            # Current position in the observation vector
            obs_pos = 0
            
            # Weekly measurement noise (standard precision)
            R[obs_pos:obs_pos+Nw, obs_pos:obs_pos+Nw] = np.eye(Nw) * 1e-3
            obs_pos += Nw
            
            # Monthly measurement noise (extreme precision to enforce constraint)
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-14
                obs_pos += Nm
            
            # Quarterly measurement noise (even more extreme precision)
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-14
                obs_pos += Nq
            
            # Add VAR-latent alignment constraints outside month/quarter end if included
            if not monthly_constraints_added and t % 5 == 0:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
            if not quarterly_constraints_added and t % 10 == 0:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
            
            # UPDATE STEP WITH NUMERICAL STABILITY
            # ------------------------------
            y_hat = H @ a_pred
            nu = y - y_hat  # Innovation
            
            # Innovation covariance - with careful regularization
            S = H @ P_pred @ H.T + R
            S = 0.5 * (S + S.T)  # Ensure perfect symmetry
            
            # Add regularization more safely
            S_reg = S.copy()
            try:
                # Use eigvalsh for symmetric matrices which returns real eigenvalues
                eig_vals = np.linalg.eigvalsh(S)
                min_eig = np.min(eig_vals)
                if min_eig < 1e-10:
                    S_reg += np.eye(S.shape[0]) * (1e-10 - min_eig)
            except:
                # Fallback to simple regularization if eigvalsh fails
                S_reg += np.eye(S.shape[0]) * 1e-8
                if j == 0 and t < 10:
                    print(f"Warning: Using diagonal regularization for S at t={t}")
            
            try:
                # Kalman gain
                K = P_pred @ H.T @ invert_matrix(S_reg)
                
                # Update state and covariance
                a_t = a_pred + K @ nu
                P_t = P_pred - K @ H @ P_pred
                
                # Force symmetry more carefully
                P_t = 0.5 * (P_t + P_t.T)  # Ensure perfect symmetry
                
                # Force positive definiteness more robustly
                try:
                    # Use eigvalsh for symmetric matrices
                    eig_vals = np.linalg.eigvalsh(P_t)
                    min_eig = np.min(eig_vals)
                    if min_eig < 1e-8:
                        P_t += np.eye(P_t.shape[0]) * (1e-8 - min_eig)
                except:
                    # If eigvalsh fails, use a more brute-force approach
                    P_t += np.eye(P_t.shape[0]) * 1e-6
                    if j == 0 and t < 10:
                        print(f"Warning: Using diagonal regularization for P_t at t={t}")
                
                # ENHANCED MEASUREMENT EQUATION ENFORCEMENT
                # ------------------------------------------------
                
                # Month-end constraint enforcement using the measurement equation
                if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                    for m in range(Nm):
                        # Get observed monthly value
                        m_obs = YM[m_idx, m]
                        var_block_idx = Nw + m
                        
                        # 1. First let's set the VAR block variable to match observation with high precision
                        # This is done through a "phantom" observation using the Kalman update
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([m_obs])
                        R_phantom = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for this phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Now enforce the temporal aggregation constraint with high precision
                        # Build measurement matrix for aggregation
                        H_agg = np.zeros((1, nstate))
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rmw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation for aggregation
                        y_agg = np.array([m_obs])
                        R_agg = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, monthly_start + m] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping afterward to prevent extreme values
                        threshold = 3.0
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use a more conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * (w / (rmw-1))
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(m_obs) * pattern_factor)
                        
                        # 5. Verify the constraint is reasonably enforced (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        else:
                            agg_value = np.sum([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - m_obs)
                            print(f"  Month {m_idx+1}, Var {m+1}: Target={m_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # Quarter-end constraint enforcement using measurement equation - similar approach
                if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                    for q in range(Nq):
                        # Get observed quarterly value
                        q_obs = YQ[q_idx, q]
                        var_block_idx = Nw + Nm + q
                        
                        # 1. Set VAR block variable with high precision
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([q_obs])
                        R_phantom = np.array([[1e-12]])
                        
                        # Kalman gain for phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Enforce temporal aggregation constraint
                        H_agg = np.zeros((1, nstate))
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rqw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation
                        y_agg = np.array([q_obs])
                        R_agg = np.array([[1e-12]])
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, quarterly_start + q] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping to prevent extremes
                        threshold = 3.0
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * np.sin(np.pi * w / rqw)
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(q_obs) * pattern_factor)
                        
                        # 5. Verify constraint (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        else:
                            agg_value = np.sum([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - q_obs)
                            print(f"  Quarter {q_idx+1}, Var {q+1}: Target={q_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # ENHANCED VALUE CLAMPING
                # Apply global state value clipping
                for i in range(nstate):
                    # More aggressive clipping for latent states
                    if i >= monthly_start:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0  # Reset to zero if NaN or Inf
                        elif abs(a_t[i]) > 3.0:  # Stricter threshold
                            a_t[i] = np.sign(a_t[i]) * 3.0  # Clip to ±3.0
                            
                    # Standard clipping for VAR block
                    else:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0
                        elif abs(a_t[i]) > 5.0:
                            a_t[i] = np.sign(a_t[i]) * 5.0
                
                # ADDITIONAL GLOBAL STATE MONITORING
                # Periodically scan and reset problematic states
                if t % 10 == 0:
                    # Check for any remaining extreme values or instabilities
                    for i in range(nstate):
                        if abs(a_t[i]) > 2.0 and i >= monthly_start:
                            # For latent states, set to conservative default if still extreme
                            if i < quarterly_start:  # Monthly latent
                                m = (i - monthly_start) % Nm
                                w = (i - monthly_start) // Nm
                                if m_idx >= 0 and m_idx < YM.shape[0]:
                                    # Set to monthly value with small deviation
                                    m_val = YM[m_idx, m]
                                    a_t[i] = m_val * (0.95 + 0.1 * w / rmw)
                                else:
                                    a_t[i] = 0.5  # Conservative default
                            else:  # Quarterly latent
                                q = (i - quarterly_start) % Nq
                                w = (i - quarterly_start) // Nq
                                if q_idx >= 0 and q_idx < YQ.shape[0]:
                                    # Set to quarterly value with small deviation
                                    q_val = YQ[q_idx, q]
                                    a_t[i] = q_val * (0.95 + 0.1 * w / rqw)
                                else:
                                    a_t[i] = 0.5
                                
                            # Reduce uncertainty for reset states
                            P_t[i, i] = 1e-6
                
            except np.linalg.LinAlgError as e:
                if j == 0:
                    print(f"Warning: Matrix inversion failed at t={t}. Using prediction only. Error: {str(e)}")
                a_t = a_pred
                P_t = P_pred
                
                # Apply value clamping even when falling back to prediction
                threshold = 5.0
                for i in range(len(a_t)):
                    if np.isnan(a_t[i]) or np.isinf(a_t[i]) or abs(a_t[i]) > threshold:
                        a_t[i] = threshold * np.sign(a_t[i]) if a_t[i] != 0 else 0.001
            
            # Store filtered state and covariance
            a_filtered[t] = a_t
            P_filtered[t] = P_t
        
        # KALMAN SMOOTHER
        # -------------
        
        # Initialize smoother with last filtered state
        a_smooth = np.zeros((nobs, nstate))
        P_smooth = np.zeros((nobs, nstate, nstate))
        
        # Last state is the same for filtered and smoothed
        a_smooth[-1] = a_filtered[-1]
        P_smooth[-1] = P_filtered[-1]
        
        try:
            # Force symmetry and positive-definiteness of P_smooth[-1] before Cholesky
            P_smooth[-1] = 0.5 * (P_smooth[-1] + P_smooth[-1].T)  # Ensure symmetry
            
            try:
                # Check and fix eigenvalues
                eig_vals = np.linalg.eigvalsh(P_smooth[-1])
                min_eig = np.min(eig_vals)
                if min_eig < 1e-8:
                    P_smooth[-1] += np.eye(P_smooth[-1].shape[0]) * (1e-8 - min_eig)
            except:
                # Fallback to simple regularization
                P_smooth[-1] += np.eye(P_smooth[-1].shape[0]) * 1e-6
                if j == 0:
                    print("Warning: Using diagonal regularization for P_smooth[-1]")
            
            # Draw the last state
            Pchol = cholcovOrEigendecomp(P_smooth[-1])
            a_draw = a_smooth[-1] + Pchol @ np.random.standard_normal(nstate)
            
            # Apply value clamping to the draw - more aggressive for latent states
            threshold_var = 5.0  # VAR block
            threshold_latent = 3.0  # Latent states
            for i in range(len(a_draw)):
                if np.isnan(a_draw[i]) or np.isinf(a_draw[i]):
                    a_draw[i] = a_smooth[-1][i]  # Fall back to smoothed value
                elif i >= monthly_start:  # Latent states
                    if abs(a_draw[i]) > threshold_latent:
                        a_draw[i] = np.sign(a_draw[i]) * threshold_latent
                else:  # VAR block
                    if abs(a_draw[i]) > threshold_var:
                        a_draw[i] = np.sign(a_draw[i]) * threshold_var
            
            # Final check for aggregate constraint on last state
            last_week_idx = T0 + nobs - 1
            last_month_idx = last_week_idx // rmw
            last_quarter_idx = last_week_idx // rqw
            
            # Check if this is month-end
            is_month_end = ((last_week_idx + 1) % rmw == 0)
            if is_month_end and last_month_idx < YM.shape[0]:
                for m in range(Nm):
                    # First enforce VAR block to match observed value
                    var_block_idx = Nw + m
                    m_obs = YM[last_month_idx, m]
                    a_draw[var_block_idx] = m_obs
                    
                    # Now enforce latent state aggregation using Kalman update approach
                    H_agg = np.zeros((1, nstate))
                    for w in range(rmw):
                        state_idx = monthly_start + w*Nm + m
                        if self.temp_agg == 'mean':
                            H_agg[0, state_idx] = 1.0/rmw
                        else:  # 'sum'
                            H_agg[0, state_idx] = 1.0
                    
                    # Check current aggregation
                    m_values = np.array([a_draw[monthly_start + w*Nm + m] for w in range(rmw)])
                    if self.temp_agg == 'mean':
                        agg_value = np.mean(m_values)
                    else:
                        agg_value = np.sum(m_values)
                    
                    # If aggregation is off by more than a small tolerance, adjust the pattern
                    if abs(agg_value - m_obs) > 1e-6:
                        if self.temp_agg == 'mean':
                            # Shift pattern to match target mean while preserving shape
                            adjustment = m_obs - agg_value
                            for w in range(rmw):
                                a_draw[monthly_start + w*Nm + m] += adjustment
                        else:  # 'sum'
                            if abs(agg_value) > 1e-10:  # Avoid division by zero
                                # Scale pattern to match target sum while preserving shape
                                scale = m_obs / agg_value
                                scale = np.clip(scale, 0.5, 2.0)  # Limit scale factor
                                for w in range(rmw):
                                    a_draw[monthly_start + w*Nm + m] *= scale
                            else:
                                # If near zero, distribute evenly
                                for w in range(rmw):
                                    a_draw[monthly_start + w*Nm + m] = m_obs / rmw
            
            # Check if this is quarter-end
            is_quarter_end = ((last_week_idx + 1) % rqw == 0)
            if is_quarter_end and last_quarter_idx < YQ.shape[0]:
                for q in range(Nq):
                    # First enforce VAR block to match observed value
                    var_block_idx = Nw + Nm + q
                    q_obs = YQ[last_quarter_idx, q]
                    a_draw[var_block_idx] = q_obs
                    
                    # Now enforce latent state aggregation using Kalman update approach
                    H_agg = np.zeros((1, nstate))
                    for w in range(rqw):
                        state_idx = quarterly_start + w*Nq + q
                        if self.temp_agg == 'mean':
                            H_agg[0, state_idx] = 1.0/rqw
                        else:  # 'sum'
                            H_agg[0, state_idx] = 1.0
                    
                    # Check current aggregation
                    q_values = np.array([a_draw[quarterly_start + w*Nq + q] for w in range(rqw)])
                    if self.temp_agg == 'mean':
                        agg_value = np.mean(q_values)
                    else:
                        agg_value = np.sum(q_values)
                    
                    # If aggregation is off by more than a small tolerance, adjust the pattern
                    if abs(agg_value - q_obs) > 1e-6:
                        if self.temp_agg == 'mean':
                            # Shift pattern to match target mean while preserving shape
                            adjustment = q_obs - agg_value
                            for w in range(rqw):
                                a_draw[quarterly_start + w*Nq + q] += adjustment
                        else:  # 'sum'
                            if abs(agg_value) > 1e-10:  # Avoid division by zero
                                # Scale pattern to match target sum while preserving shape
                                scale = q_obs / agg_value
                                scale = np.clip(scale, 0.75, 1.5)  # Tighter limits for quarterly
                                for w in range(rqw):
                                    a_draw[quarterly_start + w*Nq + q] *= scale
                            else:
                                # If near zero, distribute evenly
                                for w in range(rqw):
                                    a_draw[quarterly_start + w*Nq + q] = q_obs / rqw
            
            # Store the draw
            a_draws[j, -1] = a_draw
            
            # Backward recursion
            for t in range(nobs-2, -1, -1):
                # Get filtered state and covariance
                a_filt = a_filtered[t]
                P_filt = P_filtered[t]
                
                # Predict one step ahead
                a_pred = F @ a_filt + c.flatten()
                P_pred = F @ P_filt @ F.T + Q
                P_pred = 0.5 * (P_pred + P_pred.T)  # Ensure symmetry
                
                try:
                    # Add regularization to P_pred before inversion if needed
                    P_pred_reg = P_pred.copy()
                    try:
                        eig_vals = np.linalg.eigvalsh(P_pred)
                        min_eig = np.min(eig_vals)
                        if min_eig < 1e-8:
                            P_pred_reg += np.eye(P_pred.shape[0]) * (1e-8 - min_eig)
                    except:
                        # Fallback to simple regularization
                        P_pred_reg += np.eye(P_pred.shape[0]) * 1e-6
                    
                    # Smoothing gain
                    J_t = P_filt @ F.T @ invert_matrix(P_pred_reg)
                    
                    # Smoothed mean and covariance
                    a_smooth_t = a_filt + J_t @ (a_draw - a_pred)
                    P_smooth_t = P_filt - J_t @ (P_pred - P_smooth[t+1]) @ J_t.T
                    P_smooth_t = 0.5 * (P_smooth_t + P_smooth_t.T)  # Ensure symmetry
                    
                    # Force positive definiteness
                    try:
                        eig_vals = np.linalg.eigvalsh(P_smooth_t)
                        min_eig = np.min(eig_vals)
                        if min_eig < 1e-8:
                            P_smooth_t += np.eye(P_smooth_t.shape[0]) * (1e-8 - min_eig)
                    except:
                        # Fallback
                        P_smooth_t += np.eye(P_smooth_t.shape[0]) * 1e-6
                    
                    # Store smoothed state and covariance
                    a_smooth[t] = a_smooth_t
                    P_smooth[t] = P_smooth_t
                    
                    # Draw state
                    Pchol = cholcovOrEigendecomp(P_smooth_t)
                    a_draw = a_smooth_t + Pchol @ np.random.standard_normal(nstate)
                    
                    # Apply value clamping to the draw - more aggressive for latent states
                    threshold_var = 5.0  # VAR block
                    threshold_latent = 3.0  # Latent states
                    for i in range(len(a_draw)):
                        if np.isnan(a_draw[i]) or np.isinf(a_draw[i]):
                            a_draw[i] = a_smooth_t[i]  # Fall back to smoothed value
                        elif i >= monthly_start:  # Latent states
                            if abs(a_draw[i]) > threshold_latent:
                                a_draw[i] = np.sign(a_draw[i]) * threshold_latent
                        else:  # VAR block
                            if abs(a_draw[i]) > threshold_var:
                                a_draw[i] = np.sign(a_draw[i]) * threshold_var
                    
                    # Check and enforce constraints for this period
                    w_idx = T0 + t
                    m_idx = w_idx // rmw
                    q_idx = w_idx // rqw
                    
                    # Check if this is month-end
                    is_month_end = ((w_idx + 1) % rmw == 0)
                    if is_month_end and m_idx < YM.shape[0]:
                        for m in range(Nm):
                            # First enforce VAR block to match observed value
                            var_block_idx = Nw + m
                            m_obs = YM[m_idx, m]
                            a_draw[var_block_idx] = m_obs
                            
                            # Now enforce latent state aggregation using Kalman update approach
                            # Check current aggregation
                            m_values = np.array([a_draw[monthly_start + w*Nm + m] for w in range(rmw)])
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(m_values)
                            else:
                                agg_value = np.sum(m_values)
                            
                            # If aggregation is off by more than a small tolerance, adjust the pattern
                            if abs(agg_value - m_obs) > 1e-6:
                                if self.temp_agg == 'mean':
                                    # Shift pattern to match target mean while preserving shape
                                    adjustment = m_obs - agg_value
                                    for w in range(rmw):
                                        a_draw[monthly_start + w*Nm + m] += adjustment
                                else:  # 'sum'
                                    if abs(agg_value) > 1e-10:  # Avoid division by zero
                                        # Scale pattern to match target sum while preserving shape
                                        scale = m_obs / agg_value
                                        scale = np.clip(scale, 0.5, 2.0)  # Limit scale factor
                                        for w in range(rmw):
                                            a_draw[monthly_start + w*Nm + m] *= scale
                                    else:
                                        # If near zero, distribute evenly
                                        for w in range(rmw):
                                            a_draw[monthly_start + w*Nm + m] = m_obs / rmw
                    
                    # Check if this is quarter-end
                    is_quarter_end = ((w_idx + 1) % rqw == 0)
                    if is_quarter_end and q_idx < YQ.shape[0]:
                        for q in range(Nq):
                            # First enforce VAR block to match observed value
                            var_block_idx = Nw + Nm + q
                            q_obs = YQ[q_idx, q]
                            a_draw[var_block_idx] = q_obs
                            
                            # Now enforce latent state aggregation using Kalman update approach
                            # Check current aggregation
                            q_values = np.array([a_draw[quarterly_start + w*Nq + q] for w in range(rqw)])
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(q_values)
                            else:
                                agg_value = np.sum(q_values)
                            
                            # If aggregation is off by more than a small tolerance, adjust the pattern
                            if abs(agg_value - q_obs) > 1e-6:
                                if self.temp_agg == 'mean':
                                    # Shift pattern to match target mean while preserving shape
                                    adjustment = q_obs - agg_value
                                    for w in range(rqw):
                                        a_draw[quarterly_start + w*Nq + q] += adjustment
                                else:  # 'sum'
                                    if abs(agg_value) > 1e-10:  # Avoid division by zero
                                        # Scale pattern to match target sum while preserving shape
                                        scale = q_obs / agg_value
                                        scale = np.clip(scale, 0.75, 1.5)  # Tighter limits for quarterly
                                        for w in range(rqw):
                                            a_draw[quarterly_start + w*Nq + q] *= scale
                                    else:
                                        # If near zero, distribute evenly
                                        for w in range(rqw):
                                            a_draw[quarterly_start + w*Nq + q] = q_obs / rqw
                    
                    # Also enforce VAR-to-latent alignment for first week of each period
                    if (w_idx % rmw == 0) and (w_idx // rmw) < YM.shape[0]:
                        for m in range(Nm):
                            # Force first week to match VAR block
                            a_draw[monthly_start + m] = a_draw[Nw + m]
                    
                    if (w_idx % rqw == 0) and (w_idx // rqw) < YQ.shape[0]:
                        for q in range(Nq):
                            # Force first week to match VAR block
                            a_draw[quarterly_start + q] = a_draw[Nw + Nm + q]
                    
                    # Store the draw
                    a_draws[j, t] = a_draw
                    
                except Exception as e:
                    # Fallback for smoother error
                    if j == 0:
                        print(f"Smoother error at t={t}: {str(e)}")
                    a_smooth[t] = a_filtered[t]
                    P_smooth[t] = P_filtered[t]
                    
                    # Use filtered state with small noise but apply value clamping
                    a_draws[j, t] = a_filtered[t] + np.random.standard_normal(nstate) * 1e-4
                    
                    # Apply value clamping
                    threshold = 5.0
                    for i in range(len(a_draws[j, t])):
                        if np.isnan(a_draws[j, t, i]) or np.isinf(a_draws[j, t, i]) or abs(a_draws[j, t, i]) > threshold:
                            a_draws[j, t, i] = threshold * np.sign(a_draws[j, t, i]) if a_draws[j, t, i] != 0 else 0.001
            
        except Exception as e:
            # Fallback for smoother initialization error
            if j == 0:
                print(f"Smoother initialization error: {str(e)}")
            
            # Copy filtered states but apply value clamping
            for t in range(nobs):
                a_draws[j, t] = a_filtered[t]
                # Apply value clamping
                threshold = 5.0
                for i in range(len(a_draws[j, t])):
                    if np.isnan(a_draws[j, t, i]) or np.isinf(a_draws[j, t, i]) or abs(a_draws[j, t, i]) > threshold:
                        a_draws[j, t, i] = threshold * np.sign(a_draws[j, t, i]) if a_draws[j, t, i] != 0 else 0.001
            
        # FINAL VERIFICATION FOR THIS DRAW
        # ------------------------------
        
        # Verify key constraints for debugging
        if j == 0 or j == self.nsim - 1:
            constraint_errors = 0
            
            # Check all month-ends
            for t in range(nobs):
                w_idx = T0 + t
                # Check if this is month-end
                if (w_idx + 1) % rmw == 0:
                    m_idx = w_idx // rmw
                    if m_idx < YM.shape[0]:
                        for m in range(Nm):
                            # Get latent states
                            m_values = np.zeros(rmw)
                            for w in range(rmw):
                                m_values[w] = a_draws[j, t, monthly_start + w*Nm + m]
                            
                            # Check aggregation
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(m_values)
                            else:
                                agg_value = np.sum(m_values)
                            
                            error = abs(agg_value - YM[m_idx, m])
                            if error > 1e-6:
                                constraint_errors += 1
                                if constraint_errors <= 5:  # Limit output
                                    print(f"Draw {j}, Month {m_idx+1}, Var {m+1}: Agg={agg_value:.8f}, "
                                          f"Target={YM[m_idx, m]:.8f}, Error={error:.8f}")
            
            # Check all quarter-ends
            for t in range(nobs):
                w_idx = T0 + t
                # Check if this is quarter-end
                if (w_idx + 1) % rqw == 0:
                    q_idx = w_idx // rqw
                    if q_idx < YQ.shape[0]:
                        for q in range(Nq):
                            # Get latent states
                            q_values = np.zeros(rqw)
                            for w in range(rqw):
                                q_values[w] = a_draws[j, t, quarterly_start + w*Nq + q]
                            
                            # Check aggregation
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(q_values)
                            else:
                                agg_value = np.sum(q_values)
                            
                            error = abs(agg_value - YQ[q_idx, q])
                            if error > 1e-6:
                                constraint_errors += 1
                                if constraint_errors <= 5:  # Limit output
                                    print(f"Draw {j}, Quarter {q_idx+1}, Var {q+1}: Agg={agg_value:.8f}, "
                                          f"Target={YQ[q_idx, q]:.8f}, Error={error:.8f}")
            
            if constraint_errors > 0:
                print(f"Draw {j}: Found {constraint_errors} constraint violations")
            else:
                print(f"Draw {j}: All aggregation constraints satisfied")
        
        # CALCULATE VAR POSTERIOR USING ALL VARIABLES
        # ---------------------------------------
        
        # Create a combined matrix of all variables for VAR estimation
        combined_smooth = np.zeros((nobs, Ntotal))
        
        # Weekly variables (direct)
        combined_smooth[:, :Nw] = a_draws[j, :, :Nw]
        
        # Monthly variables (from state vector)
        combined_smooth[:, Nw:Nw+Nm] = a_draws[j, :, Nw:Nw+Nm]
        
        # Quarterly variables (from state vector)
        combined_smooth[:, Nw+Nm:Ntotal] = a_draws[j, :, Nw+Nm:Ntotal]
        
        # Use the combined matrix for VAR estimation
        YY = combined_smooth
        
        # Prepare lagged data for the VAR
        Z = np.zeros((nobs, Ntotal * p))
        for i in range(nobs):
            for lag in range(p):
                if i - lag >= 0:
                    Z[i, lag*Ntotal:(lag+1)*Ntotal] = YY[i-lag]
        
        # Compute actual observations for Minnesota prior
        nobs_ = YY.shape[0] - p  # Adjusted for lags
        spec = np.hstack((p, p, self.nex, Ntotal, nobs_))  # Now using all variables (Ntotal)
        
        # Calculate dummy observations for the full VAR
        YYact, YYdum, XXact, XXdum = calc_yyact(self.hyp, YY, spec)
        
        # Store simulation results for all variables
        if (j % self.thining == 0):
            j_temp = int(j/self.thining)
            if j == 0:
                # Initialize storage for all variables
                YYactsim_list = [np.zeros((math.ceil((self.nsim)/self.thining), rmw, Ntotal))]
                XXactsim_list = [np.zeros((math.ceil((self.nsim)/self.thining), rmw, Ntotal*p+1))]  # For all variables
            
            # Store the combined data
            YYactsim_list[0][j_temp, :, :] = combined_smooth[-rmw:, :]
            
            # Create lagged data matrix for all variables
            X_combined = np.zeros((rmw, Ntotal*p+1))
            for i in range(rmw):
                for lag in range(p):
                    t_idx = nobs - rmw + i - lag
                    if t_idx >= 0:
                        X_combined[i, lag*Ntotal:(lag+1)*Ntotal] = combined_smooth[t_idx, :]
            X_combined[:, -1] = 1.0  # Add constant
            
            XXactsim_list[0][j_temp, :, :] = X_combined
        
        # VAR POSTERIOR SAMPLING
        # -------------------
        
        try:
            # Standard VAR posterior calculations
            Tdummy, n = YYdum.shape
            n = int(n)
            Tdummy = int(Tdummy)
            Tobs, n = YYact.shape
            X = np.vstack((XXact, XXdum))
            Y = np.vstack((YYact, YYdum))
            T = Tobs + Tdummy
            
            # Compute posterior parameters
            vl, d, vr = np.linalg.svd(X, full_matrices=False)
            vr = vr.T
            di = 1/d
            B = vl.T @ Y
            xxi = (vr * np.tile(di.T, (Ntotal*p+1, 1)))
            inv_x = xxi @ xxi.T
            Phi_tilde = xxi @ B
            
            Sigma = (Y - X @ Phi_tilde).T @ (Y - X @ Phi_tilde)
            
            # Ensure Sigma is symmetric positive definite before inverse Wishart draw
            Sigma = 0.5 * (Sigma + Sigma.T)
            
            try:
                # Check and fix eigenvalues if needed
                eig_vals = np.linalg.eigvalsh(Sigma)
                min_eig = np.min(eig_vals)
                if min_eig < 1e-8:
                    Sigma += np.eye(Sigma.shape[0]) * (1e-8 - min_eig)
            except:
                # Fallback to simple regularization
                Sigma += np.eye(Sigma.shape[0]) * 1e-6
                if j == 0:
                    print("Warning: Using diagonal regularization for Sigma")
            
            # Draw from inverse Wishart for covariance matrix
            sigma_w = invwishart.rvs(scale=Sigma, df=T-Ntotal*p-1)
            
            # Draw VAR coefficients and check stability
            attempts = 0
            while attempts < 1000:
                try:
                    # Compute Cholesky safely
                    kron_matrix = np.kron(sigma_w, inv_x)
                    kron_matrix = 0.5 * (kron_matrix + kron_matrix.T)  # Ensure symmetry
                    
                    try:
                        # Check eigenvalues
                        eig_vals = np.linalg.eigvalsh(kron_matrix)
                        min_eig = np.min(eig_vals)
                        if min_eig < 1e-8:
                            kron_matrix += np.eye(kron_matrix.shape[0]) * (1e-8 - min_eig)
                    except:
                        # Fallback
                        kron_matrix += np.eye(kron_matrix.shape[0]) * 1e-6
                    
                    sigma_chol = cholcovOrEigendecomp(kron_matrix)
                    phi_new = np.squeeze(Phi_tilde.reshape(Ntotal*(Ntotal*p+1), 1, order="F")) + sigma_chol @ np.random.standard_normal(sigma_chol.shape[0])
                    Phi_w = phi_new.reshape(Ntotal*p+1, Ntotal, order="F")
                    
                    # NEW: Apply regularization to prevent explosive estimates
                    # Shrink coefficients for lagged variables toward zero
                    for lag in range(1, p+1):
                        lag_idx_start = (lag-1) * Ntotal
                        lag_idx_end = lag * Ntotal
                        shrinkage_factor = 0.95 ** lag  # More shrinkage for more distant lags
                        Phi_w[lag_idx_start:lag_idx_end, :] *= shrinkage_factor

                    # Constrain diagonal elements of first lag to reasonable range
                    for i in range(Ntotal):
                        # First lag diagonal elements shouldn't be too extreme
                        if abs(Phi_w[i, i]) > 0.9:
                            Phi_w[i, i] = 0.9 * np.sign(Phi_w[i, i])
                    
                    if not is_explosive(Phi_w, Ntotal, p):
                        break
                except Exception as e:
                    if j == 0 and attempts == 0:
                        print(f"VAR coefficient sampling error: {str(e)}, retrying...")
                
                attempts += 1
            
            if attempts == 1000:
                explosive_counter += 1
                print(f"Explosive VAR detected {explosive_counter} times.")
                continue
            
            # Store posterior draws
            if (j % self.thining == 0):
                j_temp = int(j/self.thining)
                Sigmap[j_temp, :, :] = sigma_w
                Phip[j_temp, :, :] = Phi_w
                Cons[j_temp, :] = Phi_w[-1, :]
                valid_draws.append(j_temp)
            
            # Update transition matrix for next iteration - now for all variables
            F[:Ntotal, :Ntotal*p] = Phi_w[:-1, :].T  # All VAR coefficients (excluding constant)
            c[:Ntotal] = np.atleast_2d(Phi_w[-1, :]).T  # Constants for all variables
            
            # Remember to keep the block structure intact for variable lags
            for i in range(p-1):
                F[Ntotal*(i+1):Ntotal*(i+2), Ntotal*i:Ntotal*(i+1)] = np.eye(Ntotal)
            
            # Monthly block shifting for latent states
            for i in range(rmw-1):
                src_pos = monthly_start + i*Nm
                dest_pos = monthly_start + (i+1)*Nm
                F[dest_pos:dest_pos+Nm, src_pos:src_pos+Nm] = np.eye(Nm)
            
            # Quarterly block shifting for latent states
            for i in range(rqw-1):
                src_pos = quarterly_start + i*Nq
                dest_pos = quarterly_start + (i+1)*Nq
                F[dest_pos:dest_pos+Nq, src_pos:src_pos+Nq] = np.eye(Nq)
            
            # Weekly influence for latent states
            F[monthly_start:monthly_start+Nm, :Nw] = 0.1 * np.eye(Nm, Nw)
            F[quarterly_start:quarterly_start+Nq, :Nw] = 0.05 * np.eye(Nq, Nw)
            
            # Update system covariance
            Q[:Ntotal, :Ntotal] = sigma_w
            

SyntaxError: incomplete input (<ipython-input-8-210a0cdbe1db>, line 954)

In [ ]:
        if j > 0:
            a_t = a_draws[j-1, -1]
            P_t = P_filtered[-1]
        
        # Kalman filter loop through all periods
        for t in range(nobs):
            # Current week index
            w_idx = T0 + t
            
            # PREDICTION STEP
            # --------------
            a_pred = F @ a_t + c.flatten()
            P_pred = F @ P_t @ F.T + Q
            P_pred = 0.5 * (P_pred + P_pred.T)  # Ensure perfect symmetry
            
            # DETERMINE AVAILABLE OBSERVATIONS
            # -----------------------------
            
            # Check which observations are available at this period
            is_quarter_end = q_obs_periods[w_idx] if w_idx < len(q_obs_periods) else False
            is_month_end = m_obs_periods[w_idx] if w_idx < len(m_obs_periods) else False
            
            # Get indices for the observation vectors
            q_idx = w_idx // rqw if is_quarter_end else -1
            m_idx = w_idx // rmw if is_month_end else -1
            
            # Debug output for first few periods
            if j == 0 and t < 5:
                print(f"Period {t+1}: week_idx={w_idx}, quarter_end={is_quarter_end}, month_end={is_month_end}")
                if is_quarter_end:
                    print(f"  Quarterly observation: {YQ[q_idx]}")
                if is_month_end:
                    print(f"  Monthly observation: {YM[m_idx]}")
            
            # BUILD ENHANCED MEASUREMENT MATRICES AND OBSERVATION VECTOR
            # -----------------------------------------------------
            
            # Start with weekly observations (always available)
            H_matrices = [H_w]
            y_obs = [YW[w_idx]]
            
            # Add monthly observations if available
            monthly_constraints_added = False
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                H_matrices.append(H_m)
                y_obs.append(YM[m_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))  # Zero difference means equality constraint
                monthly_constraints_added = True
            
            # Add quarterly observations if available
            quarterly_constraints_added = False
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                H_matrices.append(H_q)
                y_obs.append(YQ[q_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))  # Zero difference means equality constraint
                quarterly_constraints_added = True
            
            # Also enforce VAR-latent alignment constraints at every period
            # (not just at month/quarter end)
            if not monthly_constraints_added and t % 5 == 0:  # Every 5 periods to avoid too much constraint
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))
            
            if not quarterly_constraints_added and t % 10 == 0:  # Less frequent for quarterly
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))
            
            # Combined measurement matrix and observation vector
            H = np.vstack(H_matrices)
            y = np.concatenate(y_obs)
            
            # DIFFERENT MEASUREMENT NOISE BY FREQUENCY - EXTREMELY PRECISE FOR CONSTRAINTS
            # ---------------------------------------------------------------------
            
            # Create measurement noise matrix with precision scaled by frequency
            R = np.zeros((len(y), len(y)))
            
            # Current position in the observation vector
            obs_pos = 0
            
            # Weekly measurement noise (standard precision)
            R[obs_pos:obs_pos+Nw, obs_pos:obs_pos+Nw] = np.eye(Nw) * 1e-3
            obs_pos += Nw
            
            # Monthly measurement noise (extreme precision to enforce constraint)
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-14
                obs_pos += Nm
            
            # Quarterly measurement noise (even more extreme precision)
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-14
                obs_pos += Nq
            
            # Add VAR-latent alignment constraints outside month/quarter end if included
            if not monthly_constraints_added and t % 5 == 0:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
            if not quarterly_constraints_added and t % 10 == 0:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
            
            # UPDATE STEP WITH NUMERICAL STABILITY
            # ------------------------------
            y_hat = H @ a_pred
            nu = y - y_hat  # Innovation
            
            # Innovation covariance - with careful regularization
            S = H @ P_pred @ H.T + R
            S = 0.5 * (S + S.T)  # Ensure perfect symmetry
            
            # Add regularization more safely
            S_reg = S.copy()
            try:
                # Use eigvalsh for symmetric matrices which returns real eigenvalues
                eig_vals = np.linalg.eigvalsh(S)
                min_eig = np.min(eig_vals)
                if min_eig < 1e-10:
                    S_reg += np.eye(S.shape[0]) * (1e-10 - min_eig)
            except:
                # Fallback to simple regularization if eigvalsh fails
                S_reg += np.eye(S.shape[0]) * 1e-8
                if j == 0 and t < 10:
                    print(f"Warning: Using diagonal regularization for S at t={t}")
            
            try:
                # Kalman gain
                K = P_pred @ H.T @ invert_matrix(S_reg)
                
                # Update state and covariance
                a_t = a_pred + K @ nu
                P_t = P_pred - K @ H @ P_pred
                
                # Force symmetry more carefully
                P_t = 0.5 * (P_t + P_t.T)  # Ensure perfect symmetry
                
                # Force positive definiteness more robustly
                try:
                    # Use eigvalsh for symmetric matrices
                    eig_vals = np.linalg.eigvalsh(P_t)
                    min_eig = np.min(eig_vals)
                    if min_eig < 1e-8:
                        P_t += np.eye(P_t.shape[0]) * (1e-8 - min_eig)
                except:
                    # If eigvalsh fails, use a more brute-force approach
                    P_t += np.eye(P_t.shape[0]) * 1e-6
                    if j == 0 and t < 10:
                        print(f"Warning: Using diagonal regularization for P_t at t={t}")
                
                # ENHANCED MEASUREMENT EQUATION ENFORCEMENT
                # ------------------------------------------------
                
                # Month-end constraint enforcement using the measurement equation
                if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                    for m in range(Nm):
                        # Get observed monthly value
                        m_obs = YM[m_idx, m]
                        var_block_idx = Nw + m
                        
                        # 1. First let's set the VAR block variable to match observation with high precision
                        # This is done through a "phantom" observation using the Kalman update
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([m_obs])
                        R_phantom = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for this phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Now enforce the temporal aggregation constraint with high precision
                        # Build measurement matrix for aggregation
                        H_agg = np.zeros((1, nstate))
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rmw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation for aggregation
                        y_agg = np.array([m_obs])
                        R_agg = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, monthly_start + m] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping afterward to prevent extreme values
                        threshold = 3.0
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use a more conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * (w / (rmw-1))
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(m_obs) * pattern_factor)
                        
                        # 5. Verify the constraint is reasonably enforced (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        else:
                            agg_value = np.sum([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - m_obs)
                            print(f"  Month {m_idx+1}, Var {m+1}: Target={m_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # Quarter-end constraint enforcement using measurement equation - similar approach
                if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                    for q in range(Nq):
                        # Get observed quarterly value
                        q_obs = YQ[q_idx, q]
                        var_block_idx = Nw + Nm + q
                        
                        # 1. Set VAR block variable with high precision
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([q_obs])
                        R_phantom = np.array([[1e-12]])
                        
                        # Kalman gain for phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Enforce temporal aggregation constraint
                        H_agg = np.zeros((1, nstate))
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rqw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation
                        y_agg = np.array([q_obs])
                        R_agg = np.array([[1e-12]])
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, quarterly_start + q] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping to prevent extremes
                        threshold = 3.0
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * np.sin(np.pi * w / rqw)
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(q_obs) * pattern_factor)
                        
                        # 5. Verify constraint (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        else:
                            agg_value = np.sum([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - q_obs)
                            print(f"  Quarter {q_idx+1}, Var {q+1}: Target={q_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # ENHANCED VALUE CLAMPING
                # Apply global state value clipping
                for i in range(nstate):
                    # More aggressive clipping for latent states
                    if i >= monthly_start:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0  # Reset to zero if NaN or Inf
                        elif abs(a_t[i]) > 3.0:  # Stricter threshold
                            a_t[i] = np.sign(a_t[i]) * 3.0  # Clip to ±3.0
                            
                    # Standard clipping for VAR block
                    else:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0
                        elif abs(a_t[i]) > 5.0:
                            a_t[i] = np.sign(a_t[i]) * 5.0
                
                # ADDITIONAL GLOBAL STATE MONITORING
                # Periodically scan and reset problematic states
                if t % 10 == 0:
                    # Check for any remaining extreme values or instabilities
                    for i in range(nstate):
                        if abs(a_t[i]) > 2.0 and i >= monthly_start:
                            # For latent states, set to conservative default if still extreme
                            if i < quarterly_start:  # Monthly latent
                                m = (i - monthly_start) % Nm
                                w = (i - monthly_start) // Nm
                                if m_idx >= 0 and m_idx < YM.shape[0]:
                                    # Set to monthly value with small deviation
                                    m_val = YM[m_idx, m]
                                    a_t[i] = m_val * (0.95 + 0.1 * w / rmw)
                                else:
                                    a_t[i] = 0.5  # Conservative default
                            else:  # Quarterly latent
                                q = (i - quarterly_start) % Nq
                                w = (i - quarterly_start) // Nq
                                if q_idx >= 0 and q_idx < YQ.shape[0]:
                                    # Set to quarterly value with small deviation
                                    q_val = YQ[q_idx, q]
                                    a_t[i] = q_val * (0.95 + 0.1 * w / rqw)
                                else:
                                    a_t[i] = 0.5
                                
                            # Reduce uncertainty for reset states
                            P_t[i, i] = 1e-6
                
            except np.linalg.LinAlgError as e:
                if j == 0:
                    print(f"Warning: Matrix inversion failed at t={t}. Using prediction only. Error: {str(e)}")
                a_t = a_pred
                P_t = P_pred
                
                # Apply value clamping even when falling back to prediction
                threshold = 5.0
                for i in range(len(a_t)):
                    if np.isnan(a_t[i]) or np.isinf(a_t[i]) or abs(a_t[i]) > threshold:
                        a_t[i] = threshold * np.sign(a_t[i]) if a_t[i] != 0 else 0.001
            
            # Store filtered state and covariance
            a_filtered[t] = a_t
            P_filtered[t] = P_t
        
        # KALMAN SMOOTHER
        # -------------
        
        # Initialize smoother with last filtered state
        a_smooth = np.zeros((nobs, nstate))
        P_smooth = np.zeros((nobs, nstate, nstate))
        
        # Last state is the same for filtered and smoothed
        a_smooth[-1] = a_filtered[-1]
        P_smooth[-1] = P_filtered[-1]
        
        try:
            # Force symmetry and positive-definiteness of P_smooth[-1] before Cholesky
            P_smooth[-1] = 0.5 * (P_smooth[-1] + P_smooth[-1].T)  # Ensure symmetry
            
            try:
                # Check and fix eigenvalues
                eig_vals = np.linalg.eigvalsh(P_smooth[-1])
                min_eig = np.min(eig_vals)
                if min_eig < 1e-8:
                    P_smooth[-1] += np.eye(P_smooth[-1].shape[0]) * (1e-8 - min_eig)
            except:
                # Fallback to simple regularization
                P_smooth[-1] += np.eye(P_smooth[-1].shape[0]) * 1e-6
                if j == 0:
                    print("Warning: Using diagonal regularization for P_smooth[-1]")
            
            # Draw the last state
            Pchol = cholcovOrEigendecomp(P_smooth[-1])
            a_draw = a_smooth[-1] + Pchol @ np.random.standard_normal(nstate)
            
            # Apply value clamping to the draw - more aggressive for latent states
            threshold_var = 5.0  # VAR block
            threshold_latent = 3.0  # Latent states
            for i in range(len(a_draw)):
                if np.isnan(a_draw[i]) or np.isinf(a_draw[i]):
                    a_draw[i] = a_smooth[-1][i]  # Fall back to smoothed value
                elif i >= monthly_start:  # Latent states
                    if abs(a_draw[i]) > threshold_latent:
                        a_draw[i] = np.sign(a_draw[i]) * threshold_latent
                else:  # VAR block
                    if abs(a_draw[i]) > threshold_var:
                        a_draw[i] = np.sign(a_draw[i]) * threshold_var
            
            # Final check for aggregate constraint on last state
            last_week_idx = T0 + nobs - 1
            last_month_idx = last_week_idx // rmw
            last_quarter_idx = last_week_idx // rqw
            
            # Check if this is month-end
            is_month_end = ((last_week_idx + 1) % rmw == 0)
            if is_month_end and last_month_idx < YM.shape[0]:
                for m in range(Nm):
                    # First enforce VAR block to match observed value
                    var_block_idx = Nw + m
                    m_obs = YM[last_month_idx, m]
                    a_draw[var_block_idx] = m_obs
                    
                    # Now enforce latent state aggregation using Kalman update approach
                    H_agg = np.zeros((1, nstate))
                    for w in range(rmw):
                        state_idx = monthly_start + w*Nm + m
                        if self.temp_agg == 'mean':
                            H_agg[0, state_idx] = 1.0/rmw
                        else:  # 'sum'
                            H_agg[0, state_idx] = 1.0
                    
                    # Check current aggregation
                    m_values = np.array([a_draw[monthly_start + w*Nm + m] for w in range(rmw)])
                    if self.temp_agg == 'mean':
                        agg_value = np.mean(m_values)
                    else:
                        agg_value = np.sum(m_values)
                    
                    # If aggregation is off by more than a small tolerance, adjust the pattern
                    if abs(agg_value - m_obs) > 1e-6:
                        if self.temp_agg == 'mean':
                            # Shift pattern to match target mean while preserving shape
                            adjustment = m_obs - agg_value
                            for w in range(rmw):
                                a_draw[monthly_start + w*Nm + m] += adjustment
                        else:  # 'sum'
                            if abs(agg_value) > 1e-10:  # Avoid division by zero
                                # Scale pattern to match target sum while preserving shape
                                scale = m_obs / agg_value
                                scale = np.clip(scale, 0.5, 2.0)  # Limit scale factor
                                for w in range(rmw):
                                    a_draw[monthly_start + w*Nm + m] *= scale
                            else:
                                # If near zero, distribute evenly
                                for w in range(rmw):
                                    a_draw[monthly_start + w*Nm + m] = m_obs / rmw
            
            # Check if this is quarter-end
            is_quarter_end = ((last_week_idx + 1) % rqw == 0)
            if is_quarter_end and last_quarter_idx < YQ.shape[0]:
                for q in range(Nq):
                    # First enforce VAR block to match observed value
                    var_block_idx = Nw + Nm + q
                    q_obs = YQ[last_quarter_idx, q]
                    a_draw[var_block_idx] = q_obs
                    
                    # Now enforce latent state aggregation using Kalman update approach
                    H_agg = np.zeros((1, nstate))
                    for w in range(rqw):
                        state_idx = quarterly_start + w*Nq + q
                        if self.temp_agg == 'mean':
                            H_agg[0, state_idx] = 1.0/rqw
                        else:  # 'sum'
                            H_agg[0, state_idx] = 1.0
                    
                    # Check current aggregation
                    q_values = np.array([a_draw[quarterly_start + w*Nq + q] for w in range(rqw)])
                    if self.temp_agg == 'mean':
                        agg_value = np.mean(q_values)
                    else:
                        agg_value = np.sum(q_values)
                    
                    # If aggregation is off by more than a small tolerance, adjust the pattern
                    if abs(agg_value - q_obs) > 1e-6:
                        if self.temp_agg == 'mean':
                            # Shift pattern to match target mean while preserving shape
                            adjustment = q_obs - agg_value
                            for w in range(rqw):
                                a_draw[quarterly_start + w*Nq + q] += adjustment
                        else:  # 'sum'
                            if abs(agg_value) > 1e-10:  # Avoid division by zero
                                # Scale pattern to match target sum while preserving shape
                                scale = q_obs / agg_value
                                scale = np.clip(scale, 0.75, 1.5)  # Tighter limits for quarterly
                                for w in range(rqw):
                                    a_draw[quarterly_start + w*Nq + q] *= scale
                            else:
                                # If near zero, distribute evenly
                                for w in range(rqw):
                                    a_draw[quarterly_start + w*Nq + q] = q_obs / rqw
            
            # Store the draw
            a_draws[j, -1] = a_draw
            
            # Backward recursion
            for t in range(nobs-2, -1, -1):
                # Get filtered state and covariance
                a_filt = a_filtered[t]
                P_filt = P_filtered[t]
                
                # Predict one step ahead
                a_pred = F @ a_filt + c.flatten()
                P_pred = F @ P_filt @ F.T + Q
                P_pred = 0.5 * (P_pred + P_pred.T)  # Ensure symmetry
                
                try:
                    # Add regularization to P_pred before inversion if needed
                    P_pred_reg = P_pred.copy()
                    try:
                        eig_vals = np.linalg.eigvalsh(P_pred)
                        min_eig = np.min(eig_vals)
                        if min_eig < 1e-8:
                            P_pred_reg += np.eye(P_pred.shape[0]) * (1e-8 - min_eig)
                    except:
                        # Fallback to simple regularization
                        P_pred_reg += np.eye(P_pred.shape[0]) * 1e-6
                    
                    # Smoothing gain
                    J_t = P_filt @ F.T @ invert_matrix(P_pred_reg)
                    
                    # Smoothed mean and covariance
                    a_smooth_t = a_filt + J_t @ (a_draw - a_pred)
                    P_smooth_t = P_filt - J_t @ (P_pred - P_smooth[t+1]) @ J_t.T
                    P_smooth_t = 0.5 * (P_smooth_t + P_smooth_t.T)  # Ensure symmetry
                    
                    # Force positive definiteness
                    try:
                        eig_vals = np.linalg.eigvalsh(P_smooth_t)
                        min_eig = np.min(eig_vals)
                        if min_eig < 1e-8:
                            P_smooth_t += np.eye(P_smooth_t.shape[0]) * (1e-8 - min_eig)
                    except:
                        # Fallback
                        P_smooth_t += np.eye(P_smooth_t.shape[0]) * 1e-6
                    
                    # Store smoothed state and covariance
                    a_smooth[t] = a_smooth_t
                    P_smooth[t] = P_smooth_t
                    
                    # Draw state
                    Pchol = cholcovOrEigendecomp(P_smooth_t)
                    a_draw = a_smooth_t + Pchol @ np.random.standard_normal(nstate)
                    
                    # Apply value clamping to the draw - more aggressive for latent states
                    threshold_var = 5.0  # VAR block
                    threshold_latent = 3.0  # Latent states
                    for i in range(len(a_draw)):
                        if np.isnan(a_draw[i]) or np.isinf(a_draw[i]):
                            a_draw[i] = a_smooth_t[i]  # Fall back to smoothed value
                        elif i >= monthly_start:  # Latent states
                            if abs(a_draw[i]) > threshold_latent:
                                a_draw[i] = np.sign(a_draw[i]) * threshold_latent
                        else:  # VAR block
                            if abs(a_draw[i]) > threshold_var:
                                a_draw[i] = np.sign(a_draw[i]) * threshold_var
                    
                    # Check and enforce constraints for this period
                    w_idx = T0 + t
                    m_idx = w_idx // rmw
                    q_idx = w_idx // rqw
                    
                    # Check if this is month-end
                    is_month_end = ((w_idx + 1) % rmw == 0)
                    if is_month_end and m_idx < YM.shape[0]:
                        for m in range(Nm):
                            # First enforce VAR block to match observed value
                            var_block_idx = Nw + m
                            m_obs = YM[m_idx, m]
                            a_draw[var_block_idx] = m_obs
                            
                            # Now enforce latent state aggregation using Kalman update approach
                            # Check current aggregation
                            m_values = np.array([a_draw[monthly_start + w*Nm + m] for w in range(rmw)])
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(m_values)
                            else:
                                agg_value = np.sum(m_values)
                            
                            # If aggregation is off by more than a small tolerance, adjust the pattern
                            if abs(agg_value - m_obs) > 1e-6:
                                if self.temp_agg == 'mean':
                                    # Shift pattern to match target mean while preserving shape
                                    adjustment = m_obs - agg_value
                                    for w in range(rmw):
                                        a_draw[monthly_start + w*Nm + m] += adjustment
                                else:  # 'sum'
                                    if abs(agg_value) > 1e-10:  # Avoid division by zero
                                        # Scale pattern to match target sum while preserving shape
                                        scale = m_obs / agg_value
                                        scale = np.clip(scale, 0.5, 2.0)  # Limit scale factor
                                        for w in range(rmw):
                                            a_draw[monthly_start + w*Nm + m] *= scale
                                    else:
                                        # If near zero, distribute evenly
                                        for w in range(rmw):
                                            a_draw[monthly_start + w*Nm + m] = m_obs / rmw
                    
                    # Check if this is quarter-end
                    is_quarter_end = ((w_idx + 1) % rqw == 0)
                    if is_quarter_end and q_idx < YQ.shape[0]:
                        for q in range(Nq):
                            # First enforce VAR block to match observed value
                            var_block_idx = Nw + Nm + q
                            q_obs = YQ[q_idx, q]
                            a_draw[var_block_idx] = q_obs
                            
                            # Now enforce latent state aggregation using Kalman update approach
                            # Check current aggregation
                            q_values = np.array([a_draw[quarterly_start + w*Nq + q] for w in range(rqw)])
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(q_values)
                            else:
                                agg_value = np.sum(q_values)
                            
                            # If aggregation is off by more than a small tolerance, adjust the pattern
                            if abs(agg_value - q_obs) > 1e-6:
                                if self.temp_agg == 'mean':
                                    # Shift pattern to match target mean while preserving shape
                                    adjustment = q_obs - agg_value
                                    for w in range(rqw):
                                        a_draw[quarterly_start + w*Nq + q] += adjustment
                                else:  # 'sum'
                                    if abs(agg_value) > 1e-10:  # Avoid division by zero
                                        # Scale pattern to match target sum while preserving shape
                                        scale = q_obs / agg_value
                                        scale = np.clip(scale, 0.75, 1.5)  # Tighter limits for quarterly
                                        for w in range(rqw):
                                            a_draw[quarterly_start + w*Nq + q] *= scale
                                    else:
                                        # If near zero, distribute evenly
                                        for w in range(rqw):
                                            a_draw[quarterly_start + w*Nq + q] = q_obs / rqw
                    
                    # Also enforce VAR-to-latent alignment for first week of each period
                    if (w_idx % rmw == 0) and (w_idx // rmw) < YM.shape[0]:
                        for m in range(Nm):
                            # Force first week to match VAR block
                            a_draw[monthly_start + m] = a_draw[Nw + m]
                    
                    if (w_idx % rqw == 0) and (w_idx // rqw) < YQ.shape[0]:
                        for q in range(Nq):
                            # Force first week to match VAR block
                            a_draw[quarterly_start + q] = a_draw[Nw + Nm + q]
                    
                    # Store the draw
                    a_draws[j, t] = a_draw
                    
                except Exception as e:
                    # Fallback for smoother error
                    if j == 0:
                        print(f"Smoother error at t={t}: {str(e)}")
                    a_smooth[t] = a_filtered[t]
                    P_smooth[t] = P_filtered[t]
                    
                    # Use filtered state with small noise but apply value clamping
                    a_draws[j, t] = a_filtered[t] + np.random.standard_normal(nstate) * 1e-4
                    
                    # Apply value clamping
                    threshold = 5.0
                    for i in range(len(a_draws[j, t])):
                        if np.isnan(a_draws[j, t, i]) or np.isinf(a_draws[j, t, i]) or abs(a_draws[j, t, i]) > threshold:
                            a_draws[j, t, i] = threshold * np.sign(a_draws[j, t, i]) if a_draws[j, t, i] != 0 else 0.001
            
        except Exception as e:
            # Fallback for smoother initialization error
            if j == 0:
                print(f"Smoother initialization error: {str(e)}")
            
            # Copy filtered states but apply value clamping
            for t in range(nobs):
                a_draws[j, t] = a_filtered[t]
                # Apply value clamping
                threshold = 5.0
                for i in range(len(a_draws[j, t])):
                    if np.isnan(a_draws[j, t, i]) or np.isinf(a_draws[j, t, i]) or abs(a_draws[j, t, i]) > threshold:
                        a_draws[j, t, i] = threshold * np.sign(a_draws[j, t, i]) if a_draws[j, t, i] != 0 else 0.001
            
        # FINAL VERIFICATION FOR THIS DRAW
        # ------------------------------
        
        # Verify key constraints for debugging
        if j == 0 or j == self.nsim - 1:
            constraint_errors = 0
            
            # Check all month-ends
            for t in range(nobs):
                w_idx = T0 + t
                # Check if this is month-end
                if (w_idx + 1) % rmw == 0:
                    m_idx = w_idx // rmw
                    if m_idx < YM.shape[0]:
                        for m in range(Nm):
                            # Get latent states
                            m_values = np.zeros(rmw)
                            for w in range(rmw):
                                m_values[w] = a_draws[j, t, monthly_start + w*Nm + m]
                            
                            # Check aggregation
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(m_values)
                            else:
                                agg_value = np.sum(m_values)
                            
                            error = abs(agg_value - YM[m_idx, m])
                            if error > 1e-6:
                                constraint_errors += 1
                                if constraint_errors <= 5:  # Limit output
                                    print(f"Draw {j}, Month {m_idx+1}, Var {m+1}: Agg={agg_value:.8f}, "
                                          f"Target={YM[m_idx, m]:.8f}, Error={error:.8f}")
            
            # Check all quarter-ends
            for t in range(nobs):
                w_idx = T0 + t
                # Check if this is quarter-end
                if (w_idx + 1) % rqw == 0:
                    q_idx = w_idx // rqw
                    if q_idx < YQ.shape[0]:
                        for q in range(Nq):
                            # Get latent states
                            q_values = np.zeros(rqw)
                            for w in range(rqw):
                                q_values[w] = a_draws[j, t, quarterly_start + w*Nq + q]
                            
                            # Check aggregation
                            if self.temp_agg == 'mean':
                                agg_value = np.mean(q_values)
                            else:
                                agg_value = np.sum(q_values)
                            
                            error = abs(agg_value - YQ[q_idx, q])
                            if error > 1e-6:
                                constraint_errors += 1
                                if constraint_errors <= 5:  # Limit output
                                    print(f"Draw {j}, Quarter {q_idx+1}, Var {q+1}: Agg={agg_value:.8f}, "
                                          f"Target={YQ[q_idx, q]:.8f}, Error={error:.8f}")
            
            if constraint_errors > 0:
                print(f"Draw {j}: Found {constraint_errors} constraint violations")
            else:
                print(f"Draw {j}: All aggregation constraints satisfied")
        
        # CALCULATE VAR POSTERIOR USING ALL VARIABLES
        # ---------------------------------------
        
        # Create a combined matrix of all variables for VAR estimation
        combined_smooth = np.zeros((nobs, Ntotal))
        
        # Weekly variables (direct)
        combined_smooth[:, :Nw] = a_draws[j, :, :Nw]
        
        # Monthly variables (from state vector)
        combined_smooth[:, Nw:Nw+Nm] = a_draws[j, :, Nw:Nw+Nm]
        
        # Quarterly variables (from state vector)
        combined_smooth[:, Nw+Nm:Ntotal] = a_draws[j, :, Nw+Nm:Ntotal]
        
        # Use the combined matrix for VAR estimation
        YY = combined_smooth
        
        # Prepare lagged data for the VAR
        Z = np.zeros((nobs, Ntotal * p))
        for i in range(nobs):
            for lag in range(p):
                if i - lag >= 0:
                    Z[i, lag*Ntotal:(lag+1)*Ntotal] = YY[i-lag]
        
        # Compute actual observations for Minnesota prior
        nobs_ = YY.shape[0] - p  # Adjusted for lags
        spec = np.hstack((p, p, self.nex, Ntotal, nobs_))  # Now using all variables (Ntotal)
        
        # Calculate dummy observations for the full VAR
        YYact, YYdum, XXact, XXdum = calc_yyact(self.hyp, YY, spec)
        
        # Store simulation results for all variables
        if (j % self.thining == 0):
            j_temp = int(j/self.thining)
            if j == 0:
                # Initialize storage for all variables
                YYactsim_list = [np.zeros((math.ceil((self.nsim)/self.thining), rmw, Ntotal))]
                XXactsim_list = [np.zeros((math.ceil((self.nsim)/self.thining), rmw, Ntotal*p+1))]  # For all variables
            
            # Store the combined data
            YYactsim_list[0][j_temp, :, :] = combined_smooth[-rmw:, :]
            
            # Create lagged data matrix for all variables
            X_combined = np.zeros((rmw, Ntotal*p+1))
            for i in range(rmw):
                for lag in range(p):
                    t_idx = nobs - rmw + i - lag
                    if t_idx >= 0:
                        X_combined[i, lag*Ntotal:(lag+1)*Ntotal] = combined_smooth[t_idx, :]
            X_combined[:, -1] = 1.0  # Add constant
            
            XXactsim_list[0][j_temp, :, :] = X_combined
        
        # VAR POSTERIOR SAMPLING
        # -------------------
        
        try:
            # Standard VAR posterior calculations
            Tdummy, n = YYdum.shape
            n = int(n)
            Tdummy = int(Tdummy)
            Tobs, n = YYact.shape
            X = np.vstack((XXact, XXdum))
            Y = np.vstack((YYact, YYdum))
            T = Tobs + Tdummy
            
            # Compute posterior parameters
            vl, d, vr = np.linalg.svd(X, full_matrices=False)
            vr = vr.T
            di = 1/d
            B = vl.T @ Y
            xxi = (vr * np.tile(di.T, (Ntotal*p+1, 1)))
            inv_x = xxi @ xxi.T
            Phi_tilde = xxi @ B
            
            Sigma = (Y - X @ Phi_tilde).T @ (Y - X @ Phi_tilde)
            
            # Ensure Sigma is symmetric positive definite before inverse Wishart draw
            Sigma = 0.5 * (Sigma + Sigma.T)
            
            try:
                # Check and fix eigenvalues if needed
                eig_vals = np.linalg.eigvalsh(Sigma)
                min_eig = np.min(eig_vals)
                if min_eig < 1e-8:
                    Sigma += np.eye(Sigma.shape[0]) * (1e-8 - min_eig)
            except:
                # Fallback to simple regularization
                Sigma += np.eye(Sigma.shape[0]) * 1e-6
                if j == 0:
                    print("Warning: Using diagonal regularization for Sigma")
            
            # Draw from inverse Wishart for covariance matrix
            sigma_w = invwishart.rvs(scale=Sigma, df=T-Ntotal*p-1)
            
            # Draw VAR coefficients and check stability
            attempts = 0
            while attempts < 1000:
                try:
                    # Compute Cholesky safely
                    kron_matrix = np.kron(sigma_w, inv_x)
                    kron_matrix = 0.5 * (kron_matrix + kron_matrix.T)  # Ensure symmetry
                    
                    try:
                        # Check eigenvalues
                        eig_vals = np.linalg.eigvalsh(kron_matrix)
                        min_eig = np.min(eig_vals)
                        if min_eig < 1e-8:
                            kron_matrix += np.eye(kron_matrix.shape[0]) * (1e-8 - min_eig)
                    except:
                        # Fallback
                        kron_matrix += np.eye(kron_matrix.shape[0]) * 1e-6
                    
                    sigma_chol = cholcovOrEigendecomp(kron_matrix)
                    phi_new = np.squeeze(Phi_tilde.reshape(Ntotal*(Ntotal*p+1), 1, order="F")) + sigma_chol @ np.random.standard_normal(sigma_chol.shape[0])
                    Phi_w = phi_new.reshape(Ntotal*p+1, Ntotal, order="F")
                    
                    # NEW: Apply regularization to prevent explosive estimates
                    # Shrink coefficients for lagged variables toward zero
                    for lag in range(1, p+1):
                        lag_idx_start = (lag-1) * Ntotal
                        lag_idx_end = lag * Ntotal
                        shrinkage_factor = 0.95 ** lag  # More shrinkage for more distant lags
                        Phi_w[lag_idx_start:lag_idx_end, :] *= shrinkage_factor

                    # Constrain diagonal elements of first lag to reasonable range
                    for i in range(Ntotal):
                        # First lag diagonal elements shouldn't be too extreme
                        if abs(Phi_w[i, i]) > 0.9:
                            Phi_w[i, i] = 0.9 * np.sign(Phi_w[i, i])
                    
                    if not is_explosive(Phi_w, Ntotal, p):
                        break
                except Exception as e:
                    if j == 0 and attempts == 0:
                        print(f"VAR coefficient sampling error: {str(e)}, retrying...")
                
                attempts += 1
            
            if attempts == 1000:
                explosive_counter += 1
                print(f"Explosive VAR detected {explosive_counter} times.")
                continue
            
            # Store posterior draws
            if (j % self.thining == 0):
                j_temp = int(j/self.thining)
                Sigmap[j_temp, :, :] = sigma_w
                Phip[j_temp, :, :] = Phi_w
                Cons[j_temp, :] = Phi_w[-1, :]
                valid_draws.append(j_temp)
            
            # Update transition matrix for next iteration - now for all variables
            F[:Ntotal, :Ntotal*p] = Phi_w[:-1, :].T  # All VAR coefficients (excluding constant)
            c[:Ntotal] = np.atleast_2d(Phi_w[-1, :]).T  # Constants for all variables
            
            # Remember to keep the block structure intact for variable lags
            for i in range(p-1):
                F[Ntotal*(i+1):Ntotal*(i+2), Ntotal*i:Ntotal*(i+1)] = np.eye(Ntotal)
            
            # Monthly block shifting for latent states
            for i in range(rmw-1):
                src_pos = monthly_start + i*Nm
                dest_pos = monthly_start + (i+1)*Nm
                F[dest_pos:dest_pos+Nm, src_pos:src_pos+Nm] = np.eye(Nm)
            
            # Quarterly block shifting for latent states
            for i in range(rqw-1):
                src_pos = quarterly_start + i*Nq
                dest_pos = quarterly_start + (i+1)*Nq
                F[dest_pos:dest_pos+Nq, src_pos:src_pos+Nq] = np.eye(Nq)
            
            # Weekly influence for latent states
            F[monthly_start:monthly_start+Nm, :Nw] = 0.1 * np.eye(Nm, Nw)
            F[quarterly_start:quarterly_start+Nq, :Nw] = 0.05 * np.eye(Nq, Nw)
            
            # Update system covariance
            Q[:Ntotal, :Ntotal] = sigma_w
            
        except Exception as e:
            print(f"VAR posterior sampling error: {str(e)}")
            continue
    

Period 1: week_idx=12, quarter_end=False, month_end=False
Period 2: week_idx=13, quarter_end=False, month_end=False
Period 3: week_idx=14, quarter_end=False, month_end=False
Period 4: week_idx=15, quarter_end=False, month_end=True
  Monthly observation: [ 0.00543815  0.00273287 -0.00063385]
  Month 4, Var 1: Target=0.00543815, Achieved=0.00543815, Error=0.0000000004
  Month 4, Var 2: Target=0.00273287, Achieved=0.00273287, Error=0.0000000002
  Month 4, Var 3: Target=-0.00063385, Achieved=-0.00063385, Error=0.0000000001
Period 5: week_idx=16, quarter_end=False, month_end=False
  Month 5, Var 1: Target=-0.00016291, Achieved=-0.00016291, Error=0.0000000001
  Month 5, Var 2: Target=-0.00238576, Achieved=-0.00238576, Error=0.0000000001
  Month 5, Var 3: Target=-0.00158969, Achieved=-0.00158969, Error=0.0000000001
Draw 0: All aggregation constraints satisfied


SyntaxError: 'continue' not properly in loop (<ipython-input-9-0d9e7ff8e7af>, line 918)

In [ ]:
a_draw

array([ 0.06204113,  0.0147541 , -0.01632351,  0.00509658,  0.00598903,
        0.00840612,  0.00347586,  0.00078935,  0.05470422,  0.01375221,
        0.025228  ,  0.02219223,  0.04823227,  0.03777257,  0.00137952,
       -0.00305337,  0.06231899,  0.05198538, -0.13360861, -0.1378058 ,
        0.07217633,  0.03752976, -0.10447912,  0.04598964, -0.1218911 ,
       -0.00539067, -0.01774318,  0.11243958, -0.02407408, -0.08420003,
       -0.10957762,  0.01094409,  0.00743447,  0.09809498, -0.07306725,
        0.00363514, -0.05810332,  0.05794954, -0.07706872,  0.04590244,
       -0.06554533,  0.04052807,  0.0894167 , -0.00142856, -0.06498195,
       -0.10151555, -0.13808343,  0.01933834, -0.10416783,  0.12887093,
        0.06001375, -0.04759814,  0.14510398,  0.08429645, -0.04219666,
        0.02982116, -0.01102691, -0.06772813,  0.03408225,  0.17146558,
        0.06532492, -0.05431416,  0.05453679, -0.08666135,  0.20199683,
        0.02772652, -0.04708155,  0.06269514,  0.02306297, -0.09

In [ ]:
if j > 0:
            a_t = a_draws[j-1, -1]
            P_t = P_filtered[-1]
        
        # Kalman filter loop through all periods
        for t in range(nobs):
            # Current week index
            w_idx = T0 + t
            
            # PREDICTION STEP
            # --------------
            a_pred = F @ a_t + c.flatten()
            P_pred = F @ P_t @ F.T + Q
            P_pred = 0.5 * (P_pred + P_pred.T)  # Ensure perfect symmetry
            
            # DETERMINE AVAILABLE OBSERVATIONS
            # -----------------------------
            
            # Check which observations are available at this period
            is_quarter_end = q_obs_periods[w_idx] if w_idx < len(q_obs_periods) else False
            is_month_end = m_obs_periods[w_idx] if w_idx < len(m_obs_periods) else False
            
            # Get indices for the observation vectors
            q_idx = w_idx // rqw if is_quarter_end else -1
            m_idx = w_idx // rmw if is_month_end else -1
            
            # Debug output for first few periods
            if j == 0 and t < 5:
                print(f"Period {t+1}: week_idx={w_idx}, quarter_end={is_quarter_end}, month_end={is_month_end}")
                if is_quarter_end:
                    print(f"  Quarterly observation: {YQ[q_idx]}")
                if is_month_end:
                    print(f"  Monthly observation: {YM[m_idx]}")
            
            # BUILD ENHANCED MEASUREMENT MATRICES AND OBSERVATION VECTOR
            # -----------------------------------------------------
            
            # Start with weekly observations (always available)
            H_matrices = [H_w]
            y_obs = [YW[w_idx]]
            
            # Add monthly observations if available
            monthly_constraints_added = False
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                H_matrices.append(H_m)
                y_obs.append(YM[m_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))  # Zero difference means equality constraint
                monthly_constraints_added = True
            
            # Add quarterly observations if available
            quarterly_constraints_added = False
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                H_matrices.append(H_q)
                y_obs.append(YQ[q_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))  # Zero difference means equality constraint
                quarterly_constraints_added = True
            
            # Also enforce VAR-latent alignment constraints at every period
            # (not just at month/quarter end)
            if not monthly_constraints_added and t % 5 == 0:  # Every 5 periods to avoid too much constraint
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))
            
            if not quarterly_constraints_added and t % 10 == 0:  # Less frequent for quarterly
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))
            
            # Combined measurement matrix and observation vector
            H = np.vstack(H_matrices)
            y = np.concatenate(y_obs)
            
            # DIFFERENT MEASUREMENT NOISE BY FREQUENCY - EXTREMELY PRECISE FOR CONSTRAINTS
            # ---------------------------------------------------------------------
            
            # Create measurement noise matrix with precision scaled by frequency
            R = np.zeros((len(y), len(y)))
            
            # Current position in the observation vector
            obs_pos = 0
            
            # Weekly measurement noise (standard precision)
            R[obs_pos:obs_pos+Nw, obs_pos:obs_pos+Nw] = np.eye(Nw) * 1e-3
            obs_pos += Nw
            
            # Monthly measurement noise (extreme precision to enforce constraint)
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-14
                obs_pos += Nm
            
            # Quarterly measurement noise (even more extreme precision)
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-14
                obs_pos += Nq
            
            # Add VAR-latent alignment constraints outside month/quarter end if included
            if not monthly_constraints_added and t % 5 == 0:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
            if not quarterly_constraints_added and t % 10 == 0:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
            
            # UPDATE STEP WITH NUMERICAL STABILITY
            # ------------------------------
            y_hat = H @ a_pred
            nu = y - y_hat  # Innovation
            
            # Innovation covariance - with careful regularization
            S = H @ P_pred @ H.T + R
            S = 0.5 * (S + S.T)  # Ensure perfect symmetry
            
            # Add regularization more safely
            S_reg = S.copy()
            try:
                # Use eigvalsh for symmetric matrices which returns real eigenvalues
                eig_vals = np.linalg.eigvalsh(S)
                min_eig = np.min(eig_vals)
                if min_eig < 1e-10:
                    S_reg += np.eye(S.shape[0]) * (1e-10 - min_eig)
            except:
                # Fallback to simple regularization if eigvalsh fails
                S_reg += np.eye(S.shape[0]) * 1e-8
                if j == 0 and t < 10:
                    print(f"Warning: Using diagonal regularization for S at t={t}")
            
            try:
                # Kalman gain
                K = P_pred @ H.T @ invert_matrix(S_reg)
                
                # Update state and covariance
                a_t = a_pred + K @ nu
                P_t = P_pred - K @ H @ P_pred
                
                # Force symmetry more carefully
                P_t = 0.5 * (P_t + P_t.T)  # Ensure perfect symmetry
                
                # Force positive definiteness more robustly
                try:
                    # Use eigvalsh for symmetric matrices
                    eig_vals = np.linalg.eigvalsh(P_t)
                    min_eig = np.min(eig_vals)
                    if min_eig < 1e-8:
                        P_t += np.eye(P_t.shape[0]) * (1e-8 - min_eig)
                except:
                    # If eigvalsh fails, use a more brute-force approach
                    P_t += np.eye(P_t.shape[0]) * 1e-6
                    if j == 0 and t < 10:
                        print(f"Warning: Using diagonal regularization for P_t at t={t}")
                
                # ENHANCED MEASUREMENT EQUATION ENFORCEMENT
                # ------------------------------------------------
                
                # Month-end constraint enforcement using the measurement equation
                if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                    for m in range(Nm):
                        # Get observed monthly value
                        m_obs = YM[m_idx, m]
                        var_block_idx = Nw + m
                        
                        # 1. First let's set the VAR block variable to match observation with high precision
                        # This is done through a "phantom" observation using the Kalman update
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([m_obs])
                        R_phantom = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for this phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Now enforce the temporal aggregation constraint with high precision
                        # Build measurement matrix for aggregation
                        H_agg = np.zeros((1, nstate))
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rmw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation for aggregation
                        y_agg = np.array([m_obs])
                        R_agg = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, monthly_start + m] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping afterward to prevent extreme values
                        threshold = 3.0
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use a more conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * (w / (rmw-1))
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(m_obs) * pattern_factor)
                        
                        # 5. Verify the constraint is reasonably enforced (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        else:
                            agg_value = np.sum([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - m_obs)
                            print(f"  Month {m_idx+1}, Var {m+1}: Target={m_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # Quarter-end constraint enforcement using measurement equation - similar approach
                if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                    for q in range(Nq):
                        # Get observed quarterly value
                        q_obs = YQ[q_idx, q]
                        var_block_idx = Nw + Nm + q
                        
                        # 1. Set VAR block variable with high precision
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([q_obs])
                        R_phantom = np.array([[1e-12]])
                        
                        # Kalman gain for phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Enforce temporal aggregation constraint
                        H_agg = np.zeros((1, nstate))
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rqw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation
                        y_agg = np.array([q_obs])
                        R_agg = np.array([[1e-12]])
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, quarterly_start + q] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping to prevent extremes
                        threshold = 3.0
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * np.sin(np.pi * w / rqw)
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(q_obs) * pattern_factor)
                        
                        # 5. Verify constraint (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        else:
                            agg_value = np.sum([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - q_obs)
                            print(f"  Quarter {q_idx+1}, Var {q+1}: Target={q_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # ENHANCED VALUE CLAMPING
                # Apply global state value clipping
                for i in range(nstate):
                    # More aggressive clipping for latent states
                    if i >= monthly_start:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0  # Reset to zero if NaN or Inf
                        elif abs(a_t[i]) > 3.0:  # Stricter threshold
                            a_t[i] = np.sign(a_t[i]) * 3.0  # Clip to ±3.0
                            
                    # Standard clipping for VAR block
                    else:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0
                        elif abs(a_t[i]) > 5.0:
                            a_t[i] = np.sign(a_t[i]) * 5.0
                
                # ADDITIONAL GLOBAL STATE MONITORING
                # Periodically scan and reset problematic states
                if t % 10 == 0:
                    # Check for any remaining extreme values or instabilities
                    for i in range(nstate):
                        if abs(a_t[i]) > 2.0 and i >= monthly_start:
                            # For latent states, set to conservative default if still extreme
                            if i < quarterly_start:  # Monthly latent
                                m = (i - monthly_start) % Nm
                                w = (i - monthly_start) // Nm
                                if m_idx >= 0 and m_idx < YM.shape[0]:
                                    # Set to monthly value with small deviation
                                    m_val = YM[m_idx, m]
                                    a_t[i] = m_val * (0.95 + 0.1 * w / rmw)
                                else:
                                    a_t[i] = 0.5  # Conservative default
                            else:  # Quarterly latent
                                q = (i - quarterly_start) % Nq
                                w = (i - quarterly_start) // Nq
                                if q_idx >= 0 and q_idx < YQ.shape[0]:
                                    # Set to quarterly value with small deviation
                                    q_val = YQ[q_idx, q]
                                    a_t[i] = q_val * (0.95 + 0.1 * w / rqw)
                                else:
                                    a_t[i] = 0.5
                                
                            # Reduce uncertainty for reset states
                            P_t[i, i] = 1e-6
                
            except np.linalg.LinAlgError as e:
                if j == 0:
                    print(f"Warning: Matrix inversion failed at t={t}. Using prediction only. Error: {str(e)}")
                a_t = a_pred
                P_t = P_pred
                
                # Apply value clamping even when falling back to prediction
                threshold = 5.0
                for i in range(len(a_t)):
                    if np.isnan(a_t[i]) or np.isinf(a_t[i]) or abs(a_t[i]) > threshold:
                        a_t[i] = threshold * np.sign(a_t[i]) if a_t[i] != 0 else 0.001
            
            # Store filtered state and covariance
            a_filtered[t] = a_t
            P_filtered[t] = P_t
        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [ ]:
        if j > 0:
            a_t = a_draws[j-1, -1]
            P_t = P_filtered[-1]
        
        # Kalman filter loop through all periods
        for t in range(nobs):
            # Current week index
            w_idx = T0 + t
            
            # PREDICTION STEP
            # --------------
            a_pred = F @ a_t + c.flatten()
            P_pred = F @ P_t @ F.T + Q
            P_pred = 0.5 * (P_pred + P_pred.T)  # Ensure perfect symmetry
            
            # DETERMINE AVAILABLE OBSERVATIONS
            # -----------------------------
            
            # Check which observations are available at this period
            is_quarter_end = q_obs_periods[w_idx] if w_idx < len(q_obs_periods) else False
            is_month_end = m_obs_periods[w_idx] if w_idx < len(m_obs_periods) else False
            
            # Get indices for the observation vectors
            q_idx = w_idx // rqw if is_quarter_end else -1
            m_idx = w_idx // rmw if is_month_end else -1
            
            # Debug output for first few periods
            if j == 0 and t < 5:
                print(f"Period {t+1}: week_idx={w_idx}, quarter_end={is_quarter_end}, month_end={is_month_end}")
                if is_quarter_end:
                    print(f"  Quarterly observation: {YQ[q_idx]}")
                if is_month_end:
                    print(f"  Monthly observation: {YM[m_idx]}")
            
            # BUILD ENHANCED MEASUREMENT MATRICES AND OBSERVATION VECTOR
            # -----------------------------------------------------
            
            # Start with weekly observations (always available)
            H_matrices = [H_w]
            y_obs = [YW[w_idx]]
            
            # Add monthly observations if available
            monthly_constraints_added = False
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                H_matrices.append(H_m)
                y_obs.append(YM[m_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))  # Zero difference means equality constraint
                monthly_constraints_added = True
            
            # Add quarterly observations if available
            quarterly_constraints_added = False
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                H_matrices.append(H_q)
                y_obs.append(YQ[q_idx])
                
                # Always add the constraint that links VAR block to latent states
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))  # Zero difference means equality constraint
                quarterly_constraints_added = True
            
            # Also enforce VAR-latent alignment constraints at every period
            # (not just at month/quarter end)
            if not monthly_constraints_added and t % 5 == 0:  # Every 5 periods to avoid too much constraint
                H_matrices.append(H_m_constraint)
                y_obs.append(np.zeros(Nm))
            
            if not quarterly_constraints_added and t % 10 == 0:  # Less frequent for quarterly
                H_matrices.append(H_q_constraint)
                y_obs.append(np.zeros(Nq))
            
            # Combined measurement matrix and observation vector
            H = np.vstack(H_matrices)
            y = np.concatenate(y_obs)
            
            # DIFFERENT MEASUREMENT NOISE BY FREQUENCY - EXTREMELY PRECISE FOR CONSTRAINTS
            # ---------------------------------------------------------------------
            
            # Create measurement noise matrix with precision scaled by frequency
            R = np.zeros((len(y), len(y)))
            
            # Current position in the observation vector
            obs_pos = 0
            
            # Weekly measurement noise (standard precision)
            R[obs_pos:obs_pos+Nw, obs_pos:obs_pos+Nw] = np.eye(Nw) * 1e-3
            obs_pos += Nw
            
            # Monthly measurement noise (extreme precision to enforce constraint)
            if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-14
                obs_pos += Nm
            
            # Quarterly measurement noise (even more extreme precision)
            if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
                
                # VAR-to-latent constraint noise (extremely low)
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-14
                obs_pos += Nq
            
            # Add VAR-latent alignment constraints outside month/quarter end if included
            if not monthly_constraints_added and t % 5 == 0:
                R[obs_pos:obs_pos+Nm, obs_pos:obs_pos+Nm] = np.eye(Nm) * 1e-12
                obs_pos += Nm
                
            if not quarterly_constraints_added and t % 10 == 0:
                R[obs_pos:obs_pos+Nq, obs_pos:obs_pos+Nq] = np.eye(Nq) * 1e-12
                obs_pos += Nq
            
            # UPDATE STEP WITH NUMERICAL STABILITY
            # ------------------------------
            y_hat = H @ a_pred
            nu = y - y_hat  # Innovation
            
            # Innovation covariance - with careful regularization
            S = H @ P_pred @ H.T + R
            S = 0.5 * (S + S.T)  # Ensure perfect symmetry
            
            # Add regularization more safely
            S_reg = S.copy()
            try:
                # Use eigvalsh for symmetric matrices which returns real eigenvalues
                eig_vals = np.linalg.eigvalsh(S)
                min_eig = np.min(eig_vals)
                if min_eig < 1e-10:
                    S_reg += np.eye(S.shape[0]) * (1e-10 - min_eig)
            except:
                # Fallback to simple regularization if eigvalsh fails
                S_reg += np.eye(S.shape[0]) * 1e-8
                if j == 0 and t < 10:
                    print(f"Warning: Using diagonal regularization for S at t={t}")
            
            try:
                # Kalman gain
                K = P_pred @ H.T @ invert_matrix(S_reg)
                
                # Update state and covariance
                a_t = a_pred + K @ nu
                P_t = P_pred - K @ H @ P_pred
                
                # Force symmetry more carefully
                P_t = 0.5 * (P_t + P_t.T)  # Ensure perfect symmetry
                
                # Force positive definiteness more robustly
                try:
                    # Use eigvalsh for symmetric matrices
                    eig_vals = np.linalg.eigvalsh(P_t)
                    min_eig = np.min(eig_vals)
                    if min_eig < 1e-8:
                        P_t += np.eye(P_t.shape[0]) * (1e-8 - min_eig)
                except:
                    # If eigvalsh fails, use a more brute-force approach
                    P_t += np.eye(P_t.shape[0]) * 1e-6
                    if j == 0 and t < 10:
                        print(f"Warning: Using diagonal regularization for P_t at t={t}")
                
                # ENHANCED MEASUREMENT EQUATION ENFORCEMENT
                # ------------------------------------------------
                
                # Month-end constraint enforcement using the measurement equation
                if is_month_end and m_idx >= 0 and m_idx < YM.shape[0]:
                    for m in range(Nm):
                        # Get observed monthly value
                        m_obs = YM[m_idx, m]
                        var_block_idx = Nw + m
                        
                        # 1. First let's set the VAR block variable to match observation with high precision
                        # This is done through a "phantom" observation using the Kalman update
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([m_obs])
                        R_phantom = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for this phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Now enforce the temporal aggregation constraint with high precision
                        # Build measurement matrix for aggregation
                        H_agg = np.zeros((1, nstate))
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rmw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation for aggregation
                        y_agg = np.array([m_obs])
                        R_agg = np.array([[1e-12]])  # Very low measurement noise
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, monthly_start + m] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping afterward to prevent extreme values
                        threshold = 3.0
                        for w in range(rmw):
                            state_idx = monthly_start + w*Nm + m
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use a more conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * (w / (rmw-1))
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(m_obs) * pattern_factor)
                        
                        # 5. Verify the constraint is reasonably enforced (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        else:
                            agg_value = np.sum([a_t[monthly_start + w*Nm + m] for w in range(rmw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - m_obs)
                            print(f"  Month {m_idx+1}, Var {m+1}: Target={m_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # Quarter-end constraint enforcement using measurement equation - similar approach
                if is_quarter_end and q_idx >= 0 and q_idx < YQ.shape[0]:
                    for q in range(Nq):
                        # Get observed quarterly value
                        q_obs = YQ[q_idx, q]
                        var_block_idx = Nw + Nm + q
                        
                        # 1. Set VAR block variable with high precision
                        H_phantom = np.zeros((1, nstate))
                        H_phantom[0, var_block_idx] = 1.0
                        y_phantom = np.array([q_obs])
                        R_phantom = np.array([[1e-12]])
                        
                        # Kalman gain for phantom observation
                        S_phantom = H_phantom @ P_t @ H_phantom.T + R_phantom
                        K_phantom = P_t @ H_phantom.T @ np.linalg.inv(S_phantom)
                        
                        # Apply update
                        a_t = a_t + K_phantom @ (y_phantom - H_phantom @ a_t)
                        P_t = P_t - K_phantom @ H_phantom @ P_t
                        
                        # 2. Enforce temporal aggregation constraint
                        H_agg = np.zeros((1, nstate))
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if self.temp_agg == 'mean':
                                H_agg[0, state_idx] = 1.0/rqw
                            else:  # 'sum'
                                H_agg[0, state_idx] = 1.0
                        
                        # Use phantom observation
                        y_agg = np.array([q_obs])
                        R_agg = np.array([[1e-12]])
                        
                        # Kalman gain for aggregation
                        S_agg = H_agg @ P_t @ H_agg.T + R_agg
                        K_agg = P_t @ H_agg.T @ np.linalg.inv(S_agg)
                        
                        # Apply update
                        a_t = a_t + K_agg @ (y_agg - H_agg @ a_t)
                        P_t = P_t - K_agg @ H_agg @ P_t
                        
                        # 3. Enforce first-week alignment with VAR block
                        H_align = np.zeros((1, nstate))
                        H_align[0, var_block_idx] = 1.0  # VAR block
                        H_align[0, quarterly_start + q] = -1.0  # First week
                        
                        y_align = np.array([0.0])  # Zero difference
                        R_align = np.array([[1e-12]])
                        
                        # Kalman gain for alignment
                        S_align = H_align @ P_t @ H_align.T + R_align
                        K_align = P_t @ H_align.T @ np.linalg.inv(S_align)
                        
                        # Apply update
                        a_t = a_t + K_align @ (y_align - H_align @ a_t)
                        P_t = P_t - K_align @ H_align @ P_t
                        
                        # 4. Apply value clipping to prevent extremes
                        threshold = 3.0
                        for w in range(rqw):
                            state_idx = quarterly_start + w*Nq + q
                            if np.isnan(a_t[state_idx]) or np.isinf(a_t[state_idx]) or abs(a_t[state_idx]) > threshold:
                                # Use conservative value but preserve some variability
                                pattern_factor = 0.95 + 0.1 * np.sin(np.pi * w / rqw)
                                a_t[state_idx] = np.sign(a_t[state_idx]) * min(threshold, abs(q_obs) * pattern_factor)
                        
                        # 5. Verify constraint (for debugging)
                        if self.temp_agg == 'mean':
                            agg_value = np.mean([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        else:
                            agg_value = np.sum([a_t[quarterly_start + w*Nq + q] for w in range(rqw)])
                        
                        # Print verification for first few draws
                        if j == 0 and t < 10:
                            error = abs(agg_value - q_obs)
                            print(f"  Quarter {q_idx+1}, Var {q+1}: Target={q_obs:.8f}, Achieved={agg_value:.8f}, Error={error:.10f}")
                
                # ENHANCED VALUE CLAMPING
                # Apply global state value clipping
                for i in range(nstate):
                    # More aggressive clipping for latent states
                    if i >= monthly_start:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0  # Reset to zero if NaN or Inf
                        elif abs(a_t[i]) > 3.0:  # Stricter threshold
                            a_t[i] = np.sign(a_t[i]) * 3.0  # Clip to ±3.0
                            
                    # Standard clipping for VAR block
                    else:
                        if np.isnan(a_t[i]) or np.isinf(a_t[i]):
                            a_t[i] = 0.0
                        elif abs(a_t[i]) > 5.0:
                            a_t[i] = np.sign(a_t[i]) * 5.0
                
                # ADDITIONAL GLOBAL STATE MONITORING
                # Periodically scan and reset problematic states
                if t % 10 == 0:
                    # Check for any remaining extreme values or instabilities
                    for i in range(nstate):
                        if abs(a_t[i]) > 2.0 and i >= monthly_start:
                            # For latent states, set to conservative default if still extreme
                            if i < quarterly_start:  # Monthly latent
                                m = (i - monthly_start) % Nm
                                w = (i - monthly_start) // Nm
                                if m_idx >= 0 and m_idx < YM.shape[0]:
                                    # Set to monthly value with small deviation
                                    m_val = YM[m_idx, m]
                                    a_t[i] = m_val * (0.95 + 0.1 * w / rmw)
                                else:
                                    a_t[i] = 0.5  # Conservative default
                            else:  # Quarterly latent
                                q = (i - quarterly_start) % Nq
                                w = (i - quarterly_start) // Nq
                                if q_idx >= 0 and q_idx < YQ.shape[0]:
                                    # Set to quarterly value with small deviation
                                    q_val = YQ[q_idx, q]
                                    a_t[i] = q_val * (0.95 + 0.1 * w / rqw)
                                else:
                                    a_t[i] = 0.5
                                
                            # Reduce uncertainty for reset states
                            P_t[i, i] = 1e-6
                
            except np.linalg.LinAlgError as e:
                if j == 0:
                    print(f"Warning: Matrix inversion failed at t={t}. Using prediction only. Error: {str(e)}")
                a_t = a_pred
                P_t = P_pred
                
                # Apply value clamping even when falling back to prediction
                threshold = 5.0
                for i in range(len(a_t)):
                    if np.isnan(a_t[i]) or np.isinf(a_t[i]) or abs(a_t[i]) > threshold:
                        a_t[i] = threshold * np.sign(a_t[i]) if a_t[i] != 0 else 0.001
            
            # Store filtered state and covariance
            a_filtered[t] = a_t
            P_filtered[t] = P_t
        

Period 1: week_idx=12, quarter_end=False, month_end=False
Period 2: week_idx=13, quarter_end=False, month_end=False
Period 3: week_idx=14, quarter_end=False, month_end=False
Period 4: week_idx=15, quarter_end=False, month_end=True
  Monthly observation: [ 0.00543815  0.00273287 -0.00063385]
  Month 4, Var 1: Target=0.00543815, Achieved=0.00543815, Error=0.0000000003
  Month 4, Var 2: Target=0.00273287, Achieved=0.00273287, Error=0.0000000002
  Month 4, Var 3: Target=-0.00063385, Achieved=-0.00063385, Error=0.0000000001
Period 5: week_idx=16, quarter_end=False, month_end=False
  Month 5, Var 1: Target=-0.00016291, Achieved=-0.00016291, Error=0.0000000001
  Month 5, Var 2: Target=-0.00238576, Achieved=-0.00238576, Error=0.0000000001
  Month 5, Var 3: Target=-0.00158969, Achieved=-0.00158969, Error=0.0000000001


In [ ]:
rqw

12

In [ ]:
1.0/rqw

0.08333333333333333

In [ ]:
H_agg

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25,
        0.  , 0.  , 0.25, 0.  , 0.  , 0.25, 0.  , 0.  , 0.25, 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

In [ ]:
a_filtered

array([[ 9.24938498e-03,  7.04779213e-03,  1.10641382e-02, ...,
        -5.76574804e-04, -1.87133356e-04, -3.47150157e-04],
       [ 8.00787298e-03,  4.56441861e-03,  8.50702855e-03, ...,
        -6.26571196e-04, -1.08108895e-04, -5.76617472e-04],
       [ 5.80858959e-03,  3.53774370e-03,  6.15794667e-03, ...,
        -5.19578395e-04,  2.88983520e-05, -6.26596601e-04],
       ...,
       [ 1.79240788e-02,  1.40871017e-02,  1.60752212e-02, ...,
         6.08880454e-05, -4.51271924e-04,  4.50311306e-05],
       [ 1.33104170e-02,  1.06256902e-02,  1.16744120e-02, ...,
         6.80558416e-04, -5.75247756e-04,  6.08831725e-05],
       [ 1.08875014e-02,  8.13670500e-03,  1.15817194e-02, ...,
        -3.47144199e-04,  7.47242870e-04,  6.80558416e-04]])